In [2]:
from concurrent.futures import ThreadPoolExecutor
import time
import pandas as pd
pd.set_option('display.max_colwidth', 500)


In [ ]:
%set_env ANTHROPIC_API_KEY=${ANTHROPIC_API_KEY}%set_env LANGSMITH_TRACING=true%set_env LANGSMITH_API_KEY=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%set_env TAVILY_API_KEY=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%set_env OPENAI_API_KEY=${OPENAI_API_KEY}%set_env LANGCHAIN_API_KEY=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%set_env LANGCHAIN_TRACING_V2=true%set_env LANGCHAIN_PROJECT="Multi-agent Collaboration"

In [4]:
import configfrom openai import OpenAI# llm_api_key = config.CUSTOM_API_KEY"https://api.platform.a15t.com/v1"client = OpenAI(    api_key = llm_api_key,    base_url = llm_api_url)# from langchain.chat_models import ChatOpenAIfrom langchain_openai import ChatOpenAIfrom langchain_anthropic import ChatAnthropicfrom langchain.schema import AIMessage, HumanMessage, SystemMessageimport pandas as pddef ChatAnthropicSKT(model="skt/claude-3-5-sonnet-20241022", max_tokens=100):    

In [5]:
from typing import List, Tuple, Union, Dict, Anyimport astimport reimport jsonimport globdef dataframe_to_markdown_prompt(df, max_rows=None):    # Limit rows if needed    if max_rows is not None and len(df) > max_rows:        display_df = df.head(max_rows)        truncation_note = f"\n[Note: Only showing first {max_rows} of {len(df)} rows]"    else:        display_df = df        truncation_note = ""        # Convert to markdown    df_markdown = display_df.to_markdown()        prompt = f"""    {df_markdown}    {truncation_note}    """        return promptdef replace_strings(text, replacements):    for old, new in replacements.items():        text = text.replace(old, new)            return textdef clean_segment(segment):    """    Given a segment that is expected to be quoted (i.e. begins and ends with    the same single or double quote), remove any occurrences of that quote    from the inner content.    For example, if segment is:         "에이닷 T 멤버십 쿠폰함에 "에이닷은통화요약된닷" 입력"    then the outer quotes are preserved but the inner double quotes are removed.    """    segment = segment.strip()    if len(segment) >= 2 and segment[0] in ['"', "'"] and segment[-1] == segment[0]:        q = segment[0]        # Remove inner occurrences of the quote character.        inner = segment[1:-1].replace(q, '')        return q + inner + q    return segmentdef split_key_value(text):    """    Splits text into key and value based on the first colon that appears    outside any quoted region.    If no colon is found outside quotes, the value will be returned empty.    """    in_quote = False    quote_char = ''    for i, char in enumerate(text):        if char in ['"', "'"]:            # Toggle quote state (assumes well-formed starting/ending quotes for each token)            if in_quote:                if char == quote_char:                    in_quote = False                    quote_char = ''            else:                in_quote = True                quote_char = char        elif char == ':' and not in_quote:            return text[:i], text[i+1:]    return text, ''def split_outside_quotes(text, delimiter=','):    """    Splits the input text on the given delimiter (default comma) but only    if the delimiter occurs outside of quoted segments.    Returns a list of parts.    """    parts = []    current = []    in_quote = False    quote_char = ''    for char in text:        if char in ['"', "'"]:            # When encountering a quote, toggle our state            if in_quote:                if char == quote_char:                    in_quote = False                    quote_char = ''            else:                in_quote = True                quote_char = char            current.append(char)        elif char == delimiter and not in_quote:            parts.append(''.join(current).strip())            current = []        else:            current.append(char)    if current:        parts.append(''.join(current).strip())    return partsdef clean_ill_structured_json(text):    """    Given a string that is intended to represent a JSON-like structure    but may be ill-formed (for example, it might contain nested quotes that    break standard JSON rules), attempt to “clean” it by processing each    key–value pair.        The function uses the following heuristics:      1. Split the input text into comma-separated parts (only splitting         when the comma is not inside a quoted string).      2. For each part, split on the first colon (that is outside quotes) to separate key and value.      3. For any segment that begins and ends with a quote, remove any inner occurrences         of that same quote.      4. Rejoin the cleaned key and value.        Note: This approach does not build a fully robust JSON parser. For very complex          or deeply nested ill-structured inputs further refinement would be needed.    """    # First, split the text by commas outside of quotes.    parts = split_outside_quotes(text, delimiter=',')        cleaned_parts = []    for part in parts:        # Try to split into key and value on the first colon not inside quotes.        key, value = split_key_value(part)        key_clean = clean_segment(key)        value_clean = clean_segment(value) if value.strip() != "" else ""        if value_clean:            cleaned_parts.append(f"{key_clean}: {value_clean}")        else:            cleaned_parts.append(key_clean)        # Rejoin the cleaned parts with commas (or you can use another format if desired)    return ', '.join(cleaned_parts)def repair_json(broken_json):        # json_str = broken_json.replace("'",'"')        # Fix unquoted values (like NI00001863)    json_str = re.sub(r':\s*([a-zA-Z0-9_]+)(\s*[,}])', r': "\1"\2', broken_json)        # Fix unquoted keys    json_str = re.sub(r'([{,])\s*([a-zA-Z0-9_]+):', r'\1 "\2":', json_str)        # Fix trailing commas    json_str = re.sub(r',\s*}', '}', json_str)        return json_strdef extract_json_objects(text):    # More sophisticated pattern that tries to match proper JSON syntax    pattern = r'(\{(?:[^{}]|(?:\{(?:[^{}]|(?:\{[^{}]*\}))*\}))*\})'        result = []    for match in re.finditer(pattern, text):        potential_json = match.group(0)        try:            # Try to parse and validate            # json_obj = json.loads(repair_json(potential_json))            json_obj = ast.literal_eval(clean_ill_structured_json(repair_json(potential_json)))            result.append(json_obj)        except json.JSONDecodeError:            # Not valid JSON, skip            pass        return resultdef extract_between(text, start_marker, end_marker):    start_index = text.find(start_marker)    if start_index == -1:        return None        start_index += len(start_marker)    end_index = text.find(end_marker, start_index)    if end_index == -1:        return None        return text[start_index:end_index]def extract_content(text: str, tag_name: str) -> List[str]:    pattern = f'<{tag_name}>(.*?)</{tag_name}>'    matches = re.findall(pattern, text, re.DOTALL)    return matchesdef clean_bad_text(text):    import re        if not isinstance(text, str):        return ""        # Remove URLs and emails    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)    text = re.sub(r'\S+@\S+', ' ', text)        # Keep Korean, alphanumeric, spaces, and specific punctuation    text = re.sub(r'[^\uAC00-\uD7A3\u1100-\u11FF\w\s\.\?!,]', ' ', text)        # Normalize whitespace    text = re.sub(r'\s+', ' ', text).strip()        return textdef clean_text(text):    """    Cleans text by removing special characters that don't affect fine-tuning.    Preserves important structural elements like quotes, brackets, and JSON syntax.    Specifically handles Korean text (Hangul) properly.        Args:        text (str): The input text to clean            Returns:        str: Cleaned text ready for fine-tuning    """    import re        # Preserve the basic structure by temporarily replacing important characters    # with placeholder tokens that won't be affected by cleanup        # Step 1: Temporarily replace JSON structural elements    placeholders = {        '"': "DQUOTE_TOKEN",        "'": "SQUOTE_TOKEN",        "{": "OCURLY_TOKEN",        "}": "CCURLY_TOKEN",        "[": "OSQUARE_TOKEN",        "]": "CSQUARE_TOKEN",        ":": "COLON_TOKEN",        ",": "COMMA_TOKEN"    }        for char, placeholder in placeholders.items():        text = text.replace(char, placeholder)        # Step 2: Remove problematic characters        # Remove control characters (except newlines, carriage returns, and tabs which can be meaningful)    text = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]', '', text)        # Normalize all types of newlines to \n    text = re.sub(r'\r\n|\r', '\n', text)        # Remove zero-width characters and other invisible unicode    text = re.sub(r'[\u200B-\u200D\uFEFF\u00A0]', '', text)        # MODIFIED: Keep Korean characters (Hangul) along with other useful character sets    # This regex keeps:    # - ASCII (Basic Latin): \x00-\x7F    # - Latin-1 Supplement: \u0080-\u00FF    # - Latin Extended A & B: \u0100-\u017F\u0180-\u024F    # - Greek and Coptic: \u0370-\u03FF    # - Cyrillic: \u0400-\u04FF    # - Korean Hangul Syllables: \uAC00-\uD7A3    # - Hangul Jamo (Korean alphabet): \u1100-\u11FF    # - Hangul Jamo Extended-A: \u3130-\u318F    # - Hangul Jamo Extended-B: \uA960-\uA97F    # - Hangul Compatibility Jamo: \u3130-\u318F    # - CJK symbols and punctuation: \u3000-\u303F    # - Full-width forms (often used with CJK): \uFF00-\uFFEF    # - CJK Unified Ideographs (Basic common Chinese/Japanese characters): \u4E00-\u9FFF        # Instead of removing characters, we'll define which ones to keep    allowed_chars_pattern = r'[^\x00-\x7F\u0080-\u00FF\u0100-\u024F\u0370-\u03FF\u0400-\u04FF' + \                           r'\u1100-\u11FF\u3130-\u318F\uA960-\uA97F\u3000-\u303F' + \                           r'\uAC00-\uD7A3\uFF00-\uFFEF\u4E00-\u9FFF\n\r\t ]'    text = re.sub(allowed_chars_pattern, '', text)        # Step 3: Normalize whitespace (but preserve deliberate line breaks)    text = re.sub(r'[ \t]+', ' ', text)  # Convert multiple spaces/tabs to single space        # First ensure all newlines are standardized    text = re.sub(r'\r\n|\r', '\n', text)  # Convert all newline variants to \n        # Then normalize multiple blank lines to at most two    text = re.sub(r'\n\s*\n+', '\n\n', text)  # Convert multiple newlines to at most two        # Step 4: Restore original JSON structural elements    for char, placeholder in placeholders.items():        text = text.replace(placeholder, char)        # Step 5: Fix common JSON syntax issues that might remain    # Fix spaces between quotes and colons in JSON    text = re.sub(r'"\s+:', r'":', text)        # Fix trailing commas in arrays    text = re.sub(r',\s*]', r']', text)        # Fix trailing commas in objects    text = re.sub(r',\s*}', r'}', text)        return textdef remove_control_characters(text):    if isinstance(text, str):        # Remove control characters except commonly used whitespace        return re.sub(r'[\x00-\x08\x0B-\x0C\x0E-\x1F\x7F]', '', text)    return textimport openaifrom langchain.chains import RetrievalQAfrom langchain.llms.openai import OpenAIChat  # For compatibility with newer setup# Create a custom LLM class that uses the OpenAI client directlyclass CustomOpenAI:    def __init__(self, model="skt/a.x-3-lg"):        self.model = model            def __call__(self, prompt):        response = client.chat.completions.create(            model=self.model,            messages=[                {"role": "system", "content": "You are a helpful assistant."},                {"role": "user", "content": prompt}            ],            temperature=0.1        )        return response.choices[0].message.content# Create a simple retrieval functiondef get_relevant_context(query, vectorstore, topk=5):    docs = vectorstore.similarity_search(query, k=topk)    context = "\n\n".join([doc.page_content for doc in docs])    titles = ", ".join(set([doc.metadata['title'] for doc in docs if 'title' in doc.metadata.keys()]))    return {'title':titles, 'context':context}# Create a function to combine everythingdef answer_question(query, vectorstore):    # Get relevant context    context = get_relevant_context(query, vectorstore)        # Create combined prompt    prompt = f"Answer the following question based on the provided context:\n\nContext: {context}\n\nQuestion: {query}\n\nAnswer:"        # Use OpenAI directly    custom_llm = CustomOpenAI()  # Or your preferred model    response = custom_llm(prompt)        return responsedef is_list_of_dicts(var):    # Check if the variable is a list    if not isinstance(var, list):        return False        # Check if the list is not empty and all elements are dictionaries    if not var:  # Empty list        return False            # Check that all elements are dictionaries    return all(isinstance(item, dict) for item in var)def remove_duplicate_dicts(dict_list):    result = []    seen = set()    for d in dict_list:        # Convert dictionary to a hashable tuple of items        t = tuple(sorted(d.items()))        if t not in seen:            seen.add(t)            result.append(d)    return resultdef convert_to_custom_format(json_items):    custom_format = []        for item in json_items:        item_name = item.get("item_name_in_message", "")        item_id = item.get("item_id", "")        category = item.get("category", "")                # Create custom format for each item        custom_line = f"[Item Name] {item_name} [Item ID] {item_id} [Item Category] {category}"        custom_format.append(custom_line)        return "\n".join(custom_format)

In [133]:
from rapidfuzz import fuzz, process
import re
class KoreanEntityMatcher:
    def __init__(self, min_similarity=70, ngram_size=2, min_entity_length=2, token_similarity=True):
        self.min_similarity = min_similarity
        self.ngram_size = ngram_size
        self.min_entity_length = min_entity_length
        self.token_similarity = token_similarity  # 토큰 단위 유사도 비교 옵션 추가
        self.entities = []
        self.entity_data = {}
    def build_from_list(self, entities):
        """Build entity index from a list of entities"""
        self.entities = []
        self.entity_data = {}
        for i, entity in enumerate(entities):
            if isinstance(entity, tuple) and len(entity) == 2:
                entity_name, data = entity
                self.entities.append(entity_name)
                self.entity_data[entity_name] = data
            else:
                self.entities.append(entity)
                self.entity_data[entity] = {'id': i, 'entity': entity}
        # 각 엔티티의 정규화된 형태를 저장 (검색 최적화)
        self.normalized_entities = {}
        for entity in self.entities:
            normalized = self._normalize_text(entity)
            self.normalized_entities[normalized] = entity
        # Create n-gram index for faster candidate selection
        self._build_ngram_index(n=self.ngram_size)
    def _normalize_text(self, text):
        """텍스트 정규화 - 소문자 변환, 공백 제거 등"""
        # 소문자로 변환
        text = text.lower()
        # 연속된 공백을 하나로 통일
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    def _tokenize(self, text):
        """텍스트를 토큰으로 분리 (한글, 영문, 숫자 분리)"""
        # 한글, 영문, 숫자 토큰 추출
        tokens = re.findall(r'[가-힣]+|[a-z0-9]+', self._normalize_text(text))
        return tokens
    def _build_ngram_index(self, n=2):
        """Build n-gram index optimized for Korean characters"""
        self.ngram_index = {}
        for entity in self.entities:
            # Skip entities shorter than min_entity_length
            if len(entity) < self.min_entity_length:
                continue
            # 정규화된 엔티티 사용
            normalized_entity = self._normalize_text(entity)
            # Create n-grams for the entity
            entity_chars = list(normalized_entity)  # Split into characters for proper Korean handling
            ngrams = []
            # Create character-level n-grams (better for Korean)
            for i in range(len(entity_chars) - n + 1):
                ngram = ''.join(entity_chars[i:i+n])
                ngrams.append(ngram)
            # Add entity to the index for each n-gram
            for ngram in ngrams:
                if ngram not in self.ngram_index:
                    self.ngram_index[ngram] = set()
                self.ngram_index[ngram].add(entity)
            # 토큰 기반 n-gram도 추가 (실험적)
            tokens = self._tokenize(normalized_entity)
            for token in tokens:
                if len(token) >= n:
                    token_key = f"TOKEN:{token}"
                    if token_key not in self.ngram_index:
                        self.ngram_index[token_key] = set()
                    self.ngram_index[token_key].add(entity)
    def _get_candidates(self, text, n=None):
        """Get candidate entities based on n-gram overlap (optimized for Korean)"""
        if n is None:
            n = self.ngram_size
        # 텍스트 정규화
        normalized_text = self._normalize_text(text)
        # 정규화된 텍스트가 정확히 일치하는지 확인 (빠른 경로)
        if normalized_text in self.normalized_entities:
            entity = self.normalized_entities[normalized_text]
            return [(entity, float('inf'))]  # 정확한 일치는 무한대 점수로 표시
        text_chars = list(normalized_text)  # Split into characters for proper Korean handling
        text_ngrams = set()
        # Create character-level n-grams
        for i in range(len(text_chars) - n + 1):
            ngram = ''.join(text_chars[i:i+n])
            text_ngrams.add(ngram)
        # 토큰 기반 n-gram 추가
        tokens = self._tokenize(normalized_text)
        for token in tokens:
            if len(token) >= n:
                text_ngrams.add(f"TOKEN:{token}")
        candidates = set()
        for ngram in text_ngrams:
            if ngram in self.ngram_index:
                candidates.update(self.ngram_index[ngram])
        # Prioritize candidates with multiple n-gram matches
        candidate_scores = {}
        for candidate in candidates:
            candidate_normalized = self._normalize_text(candidate)
            candidate_chars = list(candidate_normalized)
            candidate_ngrams = set()
            # 문자 n-gram
            for i in range(len(candidate_chars) - n + 1):
                ngram = ''.join(candidate_chars[i:i+n])
                candidate_ngrams.add(ngram)
            # 토큰 기반 n-gram
            candidate_tokens = self._tokenize(candidate_normalized)
            for token in candidate_tokens:
                if len(token) >= n:
                    candidate_ngrams.add(f"TOKEN:{token}")
            # n-gram 교집합 크기로 초기 점수 계산
            overlap = len(candidate_ngrams.intersection(text_ngrams))
            # 토큰 수준 유사도 보너스 점수 추가
            token_bonus = 0
            if self.token_similarity:
                query_tokens = set(tokens)
                cand_tokens = set(candidate_tokens)
                # 공통 토큰 비율 계산
                if query_tokens and cand_tokens:
                    common = query_tokens.intersection(cand_tokens)
                    token_bonus = len(common) * 2  # 토큰 일치에 높은 가중치 부여
            candidate_scores[candidate] = overlap + token_bonus
        # Return candidates sorted by n-gram overlap score
        return sorted(candidate_scores.items(), key=lambda x: x[1], reverse=True)
    def _calculate_similarity(self, text, entity):
        """다양한 유사도 측정 방법을 결합하여 더 정확한 유사도 계산"""
        normalized_text = self._normalize_text(text)
        normalized_entity = self._normalize_text(entity)
        # 정확히 일치하면 100점 반환
        if normalized_text == normalized_entity:
            return 100
        # 기본 문자열 유사도 (fuzz.ratio)
        ratio_score = fuzz.ratio(normalized_text, normalized_entity)
        # 부분 문자열 체크 (한 문자열이 다른 문자열의 부분 문자열인 경우)
        partial_score = 0
        if normalized_text in normalized_entity:
            text_len = len(normalized_text)
            entity_len = len(normalized_entity)
            partial_score = (text_len / entity_len) * 100 if entity_len > 0 else 0
        elif normalized_entity in normalized_text:
            text_len = len(normalized_text)
            entity_len = len(normalized_entity)
            partial_score = (entity_len / text_len) * 100 if text_len > 0 else 0
        # 토큰 유사도 (토큰 단위로 비교)
        token_score = 0
        if self.token_similarity:
            text_tokens = set(self._tokenize(normalized_text))
            entity_tokens = set(self._tokenize(normalized_entity))
            if text_tokens and entity_tokens:
                common_tokens = text_tokens.intersection(entity_tokens)
                all_tokens = text_tokens.union(entity_tokens)
                if all_tokens:
                    token_score = (len(common_tokens) / len(all_tokens)) * 100
        # 토큰 순서 무시 유사도 (fuzz.token_sort_ratio)
        token_sort_score = fuzz.token_sort_ratio(normalized_text, normalized_entity)
        # 토큰 집합 유사도 (fuzz.token_set_ratio)
        token_set_score = fuzz.token_set_ratio(normalized_text, normalized_entity)
        # 최종 유사도는 여러 점수의 가중 평균
        # 토큰 유사도에 높은 가중치를 부여하여 "T우주"와 "T 우주패스"의 매칭 향상
        final_score = (
            ratio_score * 0.3 +  # 기본 유사도
            max(partial_score, 0) * 0.1 +  # 부분 문자열 유사도
            token_score * 0.2 +  # 토큰 유사도
            token_sort_score * 0.2 +  # 토큰 순서 무시 유사도
            token_set_score * 0.2  # 토큰 집합 유사도
        )
        return final_score
    def find_entities(self, text, max_candidates_per_span=10):
        """Find entity matches in Korean text using fuzzy matching"""
        # Extract spans that might contain entities
        potential_spans = self._extract_korean_spans(text)
        matches = []
        for span_text, start, end in potential_spans:
            if len(span_text.strip()) < self.min_entity_length:  # Skip spans shorter than min_entity_length
                continue
            # Get candidate entities based on n-gram overlap
            candidates = self._get_candidates(span_text)
            # If no candidates found through n-gram filtering, skip
            if not candidates:
                continue
            # Limit the number of candidates to check
            top_candidates = [c[0] for c in candidates[:max_candidates_per_span]]
            # 각 후보 엔티티에 대해 개선된 유사도 계산
            scored_matches = []
            for entity in top_candidates:
                score = self._calculate_similarity(span_text, entity)
                if score >= self.min_similarity:
                    scored_matches.append((entity, score, 0))  # 호환성을 위해 3번째 매개변수 추가
            # 기존 process.extract 대신 개선된 유사도 계산 사용
            best_matches = scored_matches
            for entity, score, _ in best_matches:
                matches.append({
                    'text': span_text,
                    'matched_entity': entity,
                    'score': score,
                    'start': start,
                    'end': end,
                    'data': self.entity_data.get(entity, {})
                })
        # Sort by position in text
        matches.sort(key=lambda x: (x['start'], -x['score']))
        # Handle overlapping matches by keeping the best match
        final_matches = self._resolve_overlapping_matches(matches)
        return final_matches
    def _extract_korean_spans(self, text):
        """한국어와 영어가 혼합된 텍스트에서 엔티티일 수 있는 잠재적 텍스트 범위 추출"""
        spans = []
        min_len = self.min_entity_length
        # 1. 영문+한글 혼합 패턴 (붙여쓰기) 예: "T우주"
        for match in re.finditer(r'[a-zA-Z]+[가-힣]+(?:\s+[가-힣가-힣a-zA-Z0-9]+)*', text):
            if len(match.group(0)) >= min_len:
                spans.append((match.group(0), match.start(), match.end()))
        # 2. 영문+한글 혼합 패턴 (띄어쓰기) 예: "T 우주"
        for match in re.finditer(r'[a-zA-Z]+\s+[가-힣]+(?:\s+[가-힣가-힣a-zA-Z0-9]+)*', text):
            if len(match.group(0)) >= min_len:
                spans.append((match.group(0), match.start(), match.end()))
        # 3. 특수한 혼합 패턴 - 영문+한글+숫자 (예: "T우주365", "SK텔레콤")
        for match in re.finditer(r'[a-zA-Z]+[가-힣]+(?:[0-9]+)?', text):
            if len(match.group(0)) >= min_len:
                spans.append((match.group(0), match.start(), match.end()))
        # 4. 연속된 두 단어까지 확장 (예: "T우주 패스")
        # 영문+한글 후 공백 하나를 두고 다른 한글 단어가 나오는 패턴
        for match in re.finditer(r'[a-zA-Z]+[가-힣]+\s+[가-힣]+', text):
            if len(match.group(0)) >= min_len:
                spans.append((match.group(0), match.start(), match.end()))
        # 5. 연속된 세 단어까지 확장 (예: "T우주 멤버십 패스")
        for match in re.finditer(r'[a-zA-Z]+[가-힣]+\s+[가-힣]+\s+[가-힣]+', text):
            if len(match.group(0)) >= min_len:
                spans.append((match.group(0), match.start(), match.end()))
        # 6. 브랜드명 + 제품명 패턴 (예: "SK텔레콤 T우주")
        for match in re.finditer(r'[a-zA-Z가-힣]+(?:\s+[a-zA-Z가-힣]+){1,3}', text):
            if len(match.group(0)) >= min_len:
                spans.append((match.group(0), match.start(), match.end()))
        # 7. 숫자와 영문 결합 패턴 (숫자 space 영문 패턴, e.g. "0 day")
        for match in re.finditer(r'\d+\s+[a-zA-Z]+', text):
            if len(match.group(0)) >= min_len:
                spans.append((match.group(0), match.start(), match.end()))
        # 8. 더 일반적인 영한 혼합 패턴
        for match in re.finditer(r'[a-zA-Z가-힣0-9]+(?:\s+[a-zA-Z가-힣0-9]+)*', text):
            if len(match.group(0)) >= min_len:
                spans.append((match.group(0), match.start(), match.end()))
        # 9. 일반적인 구분자로 분리된 텍스트 조각도 추출
        for span in re.split(r'[,\.!?;:"\'…\(\)\[\]\{\}\s_/]+', text):
            if span and len(span) >= min_len:
                span_pos = text.find(span)
                if span_pos != -1:
                    spans.append((span, span_pos, span_pos + len(span)))
        return spans
    def _remove_duplicate_entities(self, matches):
        """Keep only one instance of each unique entity"""
        if not matches:
            return []
        # Dictionary to track highest-scoring match for each entity
        best_matches = {}
        for match in matches:
            entity_key = match['matched_entity']
            # If we haven't seen this entity before, or if this match has a higher score
            # than the previously saved match for this entity, save this one
            if (entity_key not in best_matches or
                match['score'] > best_matches[entity_key]['score']):
                best_matches[entity_key] = match
        # Return the best matches sorted by start position
        return sorted(best_matches.values(), key=lambda x: x['start'])
    def _resolve_overlapping_matches(self, matches, high_score_threshold=50, overlap_tolerance=0.5):
        """
        개선된 중복 매치 해결 - 점수 기반으로 매치들을 유지하거나 제거
        Parameters:
        -----------
        matches : list
            매치 목록
        high_score_threshold : int, default=50
            이 점수 이상이면 중복/포함 관계에서도 별도 매치로 고려할 임계값
        overlap_tolerance : float, default=0.5
            중복 허용 범위 (0~1), 값이 클수록 더 많은 중복을 허용
        """
        if not matches:
            return []
        # 점수 내림차순, 길이 오름차순으로 정렬 (높은 점수, 짧은 매치 우선)
        sorted_matches = sorted(matches, key=lambda x: (-x['score'], x['end'] - x['start']))
        final_matches = []
        for current_match in sorted_matches:
            current_score = current_match['score']
            current_start, current_end = current_match['start'], current_match['end']
            current_range = set(range(current_start, current_end))
            current_len = len(current_range)
            # 높은 점수의 매치는 항상 포함
            if current_score >= high_score_threshold:
                # 기존 매치들과 비교하여 너무 많은 중복이 있는지 확인
                is_too_similar = False
                for existing_match in final_matches:
                    if existing_match['score'] < high_score_threshold:
                        continue  # 낮은 점수의 기존 매치와는 비교하지 않음
                    existing_start, existing_end = existing_match['start'], existing_match['end']
                    existing_range = set(range(existing_start, existing_end))
                    # 교집합 비율 계산
                    intersection = current_range.intersection(existing_range)
                    # 현재 매치에 대한 중복 비율
                    current_overlap_ratio = len(intersection) / current_len if current_len > 0 else 0
                    # 중복 비율이 허용 범위를 초과하고, 동일한 엔티티가 아니면 추가하지 않음
                    if (current_overlap_ratio > overlap_tolerance and
                        current_match['matched_entity'] == existing_match['matched_entity']):
                        is_too_similar = True
                        break
                if not is_too_similar:
                    final_matches.append(current_match)
            else:
                # 낮은 점수의 매치는 기존 로직 적용 (중복 확인)
                should_add = True
                for existing_match in final_matches:
                    existing_start, existing_end = existing_match['start'], existing_match['end']
                    existing_range = set(range(existing_start, existing_end))
                    # 교집합 비율 계산
                    intersection = current_range.intersection(existing_range)
                    current_overlap_ratio = len(intersection) / current_len if current_len > 0 else 0
                    # 중복 비율이 허용 범위를 초과하면 추가하지 않음
                    if current_overlap_ratio > (1 - overlap_tolerance):
                        should_add = False
                        break
                if should_add:
                    final_matches.append(current_match)
        # 시작 위치별로 정렬
        final_matches.sort(key=lambda x: x['start'])
        return final_matches
def find_entities_in_text(text, entity_list, min_similarity=70, ngram_size=3, min_entity_length=2,
                         token_similarity=True, high_score_threshold=50, overlap_tolerance=0.5):
    """
    Find entity matches in text using fuzzy matching.
    Parameters:
    -----------
    text : str
        The text to search for entities
    entity_list : list
        List of entities to match against
    min_similarity : int, default=70
        Minimum similarity score (0-100) for fuzzy matching
    ngram_size : int, default=2
        Size of character n-grams to use for indexing (2 or 3 recommended for Korean)
    min_entity_length : int, default=2
        Minimum length of entities to consider (characters)
    token_similarity : bool, default=True
        Whether to use token-based similarity measures
    high_score_threshold : int, default=50
        Score threshold above which matches are always kept regardless of overlap
    overlap_tolerance : float, default=0.5
        Overlap tolerance ratio (0-1), higher values allow more overlapping matches
    Returns:
    --------
    list
        List of matched entities with position and metadata
    """
    matcher = KoreanEntityMatcher(
        min_similarity=min_similarity,
        ngram_size=ngram_size,
        min_entity_length=min_entity_length,
        token_similarity=token_similarity
    )
    matcher.build_from_list(entity_list)
    matches = matcher.find_entities(text)
    # 기존 _resolve_overlapping_matches 메서드 대신 직접 호출
    final_matches = matcher._resolve_overlapping_matches(
        matches,
        high_score_threshold=high_score_threshold,
        overlap_tolerance=overlap_tolerance
    )
    return final_matches
# Function to highlight entities in text
def highlight_entities(text, matches):
    marked_text = text
    offset = 0
    for match in sorted(matches, key=lambda x: x['start'], reverse=True):
        start = match['start'] + offset
        end = match['end'] + offset
        entity = match['matched_entity']
        score = match['score']
        marked_text = marked_text[:start] + f"[{marked_text[start:end]}→{entity} ({score:.1f}%)]" + marked_text[end:]
        offset += len(f"[→{entity} ({score:.1f}%)]") + 2
    return marked_text


In [7]:
from difflib import SequenceMatcherdef calculate_list_similarity(list1, list2):    # Handle cases where input might be a dictionary or other types    if isinstance(list1, dict):        list1 = [str(item) for item in list1.values()]    if isinstance(list2, dict):        list2 = [str(item) for item in list2.values()]        # Ensure lists contain strings    list1 = [str(item) for item in list1]    list2 = [str(item) for item in list2]        # Convert lists to sets for comparison    set1 = set(sorted(set(list1)))    set2 = set(sorted(set(list2)))        # Calculate Jaccard similarity    intersection = len(set1.intersection(set2))    union = len(set1.union(set2))        return intersection / union if union > 0 else 0def calculate_text_similarity(text1, text2):    return SequenceMatcher(None, str(text1), str(text2)).ratio()def calculate_message_info_similarity(minfo1, minfo2):    if not isinstance(minfo1, dict) or not isinstance(minfo2, dict):        return 0.0        sender_sim = calculate_text_similarity(minfo1.get('sender',''), minfo2.get('sender',''))    title_sim = calculate_text_similarity(minfo1.get('title',''), minfo2.get('title',''))    main_theme_sim = calculate_text_similarity(minfo1.get('main_theme',''), minfo2.get('main_theme',''))    period_sim = calculate_text_similarity(minfo1.get('period',''), minfo2.get('period',''))        return (sender_sim + title_sim + main_theme_sim + period_sim) / 4def calculate_metadata_similarity(minfo1, minfo2):    if not isinstance(minfo1, dict) or not isinstance(minfo2, dict):        return 0.0        campaign_id_sim = calculate_text_similarity(minfo1.get('campaign_id',''), minfo2.get('campaign_id',''))    message_type_sim = calculate_text_similarity(minfo1.get('message_type',''), minfo2.get('message_type',''))    target_response_sim = calculate_text_similarity(minfo1.get('target_response',''), minfo2.get('target_response',''))        success_metrics_similarity = calculate_list_similarity(        minfo1.get('success_metrics',[]),         minfo2.get('success_metrics',[])    )        return (campaign_id_sim + message_type_sim + target_response_sim + success_metrics_similarity) / 4def calculate_product_similarity(prod1, prod2):    if not isinstance(prod1, dict) or not isinstance(prod2, dict):        return 0.0        name_sim = calculate_text_similarity(prod1.get('name',''), prod2.get('name',''))    category_sim = calculate_text_similarity(prod1.get('category',''), prod2.get('category',''))    benefit_sim = calculate_text_similarity(prod1.get('benefit',''), prod2.get('benefit',''))    action_sim = calculate_text_similarity(prod1.get('action',''), prod2.get('action',''))    conditions_sim = calculate_text_similarity(prod1.get('conditions',''), prod2.get('conditions',''))        return (name_sim + category_sim + benefit_sim + action_sim + conditions_sim) / 5def calculate_channel_similarity(chan1, chan2):    if not isinstance(chan1, dict) or not isinstance(chan2, dict):        return 0.0        type_sim = calculate_text_similarity(chan1.get('type',''), chan2.get('type',''))    value_sim = calculate_text_similarity(chan1.get('value',''), chan2.get('value',''))    action_sim = calculate_text_similarity(chan1.get('action',''), chan2.get('action',''))    availability_sim = calculate_text_similarity(chan1.get('availability',''), chan2.get('availability',''))        return (type_sim + value_sim + action_sim + availability_sim) / 4def calculate_target_similarity(target1, target2):    if not isinstance(target1, list) or not isinstance(target2, list):        return 0.0        similarities = []    for t1 in target1:        for t2 in target2:            segment_sim = calculate_text_similarity(t1.get('segment',''), t2.get('segment',''))            characteristics_sim = calculate_text_similarity(t1.get('characteristics',''), t2.get('characteristics',''))            similarities.append((segment_sim + characteristics_sim) / 2)        return max(similarities) if similarities else 0.0def calculate_dictionary_similarity(dict1, dict2):    # Calculate similarities for each component    message_info_similarity = calculate_message_info_similarity(        dict1.get('message_info',{}),         dict2.get('message_info',{})    )        meta_similarity = calculate_metadata_similarity(        dict1.get('metadata',{}),         dict2.get('metadata',{})    )        purpose_similarity = calculate_list_similarity(        dict1.get('purpose',[]),         dict2.get('purpose',[])    )        target_similarity = calculate_target_similarity(        dict1.get('target',[]),         dict2.get('target',[])    )        # Calculate product similarity    product_similarities = []    products1 = dict1.get('product', [])    products2 = dict2.get('product', [])    for p1, p2 in zip(products1, products2):        product_similarities.append(calculate_product_similarity(p1, p2))    product_similarity = sum(product_similarities) / len(product_similarities) if product_similarities else 0.0        # Calculate channel similarity    channel_similarities = []    channels1 = dict1.get('channel', [])    channels2 = dict2.get('channel', [])    for c1, c2 in zip(channels1, channels2):        channel_similarities.append(calculate_channel_similarity(c1, c2))    channel_similarity = sum(channel_similarities) / len(channel_similarities) if channel_similarities else 0.0    # Calculate overall similarity    overall_similarity = (        message_info_similarity +         purpose_similarity +         target_similarity +         product_similarity +         channel_similarity +        meta_similarity    ) / 5        return {        'overall_similarity': overall_similarity,        'message_info_similarity': message_info_similarity,        'purpose_similarity': purpose_similarity,        'target_similarity': target_similarity,        'product_similarity': product_similarity,        'channel_similarity': channel_similarity,        'meta_similarity': meta_similarity    }

In [ ]:
# mms_pdf = pd.read_excel("./data/mms_data_250408.xlsx", engine="openpyxl")
mms_pdf = pd.read_csv("./data/mms_data_250408.csv")
mms_pdf['msg'] = mms_pdf['msg_nm']+"\n"+mms_pdf['mms_phrs']
mms_pdf = mms_pdf.groupby(["msg_nm","mms_phrs","msg"])['offer_dt'].min().reset_index(name="offer_dt")
mms_pdf = mms_pdf.reset_index()
mms_pdf = mms_pdf.astype('str')
mms_pdf.head(1)
# item_pdf = pd.read_pickle("/home/skinet/myfiles/tos_ace/data/item_info_250318.pkl")

In [111]:
# item_pdf_raw = pd.read_excel("./data/item_info_250401.xlsx", engine="openpyxl")
item_pdf_raw = pd.read_csv("./data/item_info_250401.csv")
item_pdf = item_pdf_raw.drop_duplicates(['item_nm','item_desc']).copy()
item_pdf['item_item'] = item_pdf['item_nm']+"\n"+item_pdf['item_desc']
item_pdf['item_nm_cl'] = item_pdf['item_nm'].apply(clean_text)
item_pdf['item_desc_cl'] = item_pdf['item_desc'].fillna('').astype(str).apply(clean_text)
item_pdf['item_item_cl'] = item_pdf['item_nm_cl']+"\n"+item_pdf['item_desc_cl']
entity_list = []
for row in item_pdf.to_dict('records'):
    entity_list.append((row['item_nm'], {'item_id':row['item_id'],'category':row['item_cate_ax'], 'description':row['item_desc'], 'create_dt':row['create_dt']}))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import konlpy.tag
# For Korean text preprocessing, use KoNLPy
class KoreanTokenizer:
    def __init__(self, tagger_type='Okt'):
        # Choose a Korean morphological analyzer
        # Options: Okt, Mecab, Komoran, Hannanum, Kkma
        if tagger_type == 'Okt':
            from konlpy.tag import Okt
            self.tagger = Okt()
        elif tagger_type == 'Mecab':
            from konlpy.tag import Mecab
            self.tagger = Mecab()
        # Add other taggers as needed
    def __call__(self, text):
        # Extract nouns, adjectives, and verbs (you can customize this)
        tokens = self.tagger.morphs(text)
        return tokens
# Initialize the tokenizer (Okt is generally good for most purposes)
korean_tokenizer = KoreanTokenizer(tagger_type='Okt')
tfidf_name = TfidfVectorizer(
            analyzer='char',
            ngram_range=(3,10),
            min_df=1,
            # stop_words=None,  # Don't use stop words initially
            # token_pattern=r'(?u)\b\w+\b',  # Match any word character (more permissive)
            # strip_accents='unicode',  # Handle accented characters
        )
tfidf_cate = TfidfVectorizer(
            analyzer='char',
            ngram_range=(2,4),
            min_df=1,
            # stop_words=None,  # Don't use stop words initially
            # token_pattern=r'(?u)\b\w+\b',  # Match any word character (more permissive)
            # strip_accents='unicode',  # Handle accented characters
        )
tfidf_desc = TfidfVectorizer(
            analyzer='char',
            ngram_range=(3,5),
            min_df=1,
            # stop_words=None,  # Don't use stop words initially
            # token_pattern=r'(?u)\b\w+\b',  # Match any word character (more permissive)
            # strip_accents='unicode',  # Handle accented characters
        )
# tfidf_desc = TfidfVectorizer(
#             analyzer='word',
#             ngram_range=(1,5),
#             min_df=1,
#             stop_words=None,  # Don't use stop words initially
#             token_pattern=r'(?u)\b\w+\b',  # Match any word character (more permissive)
#             strip_accents='unicode',  # Handle accented characters
#         )
# tfidf_name = TfidfVectorizer(
#     tokenizer=korean_tokenizer,
#     ngram_range=(1, 2),  # Consider both unigrams and bigrams
#     max_features=10000,  # Limit vocabulary size
#     min_df=2,            # Minimum document frequency
#     use_idf=True,
#     sublinear_tf=True    # Apply sublinear tf scaling (1 + log(tf))
# )
# tfidf_desc = TfidfVectorizer(
#     tokenizer=korean_tokenizer,
#     ngram_range=(1, 2),  # Consider both unigrams and bigrams
#     max_features=10000,  # Limit vocabulary size
#     min_df=2,            # Minimum document frequency
#     use_idf=True,
#     sublinear_tf=True    # Apply sublinear tf scaling (1 + log(tf))
# )
# tfidf_item = TfidfVectorizer(
#     tokenizer=korean_tokenizer,
#     ngram_range=(1, 2),  # Consider both unigrams and bigrams
#     max_features=10000,  # Limit vocabulary size
#     min_df=2,            # Minimum document frequency
#     use_idf=True,
#     sublinear_tf=True    # Apply sublinear tf scaling (1 + log(tf))
# )
fs_model = "gpt"
# few_shot_pdf = pd.read_pickle("./data/few_shot_data_chat_250410_rd_500.pkl")
few_shot_pdf = pd.read_csv(f"./data/few_shot_data_{fs_model}_250415_rd_500.csv")
few_shot_pdf = few_shot_pdf.astype('str')
tfidf_matrix_name = tfidf_name.fit_transform(item_pdf['item_nm_cl'])
tfidf_matrix_cate = tfidf_cate.fit_transform(item_pdf['item_cate_ax'])
tfidf_matrix_desc = tfidf_desc.fit_transform(few_shot_pdf["msg_body"])
# tfidf_matrix_item = tfidf_item.fit_transform(item_pdf['item_item_cl'])
# import torch
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# emb_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')  # Or other appropriate model
# emb_model = emb_model.to(device)
# item_all = item_pdf['item_item_cl'].tolist()
# item_embeddings = emb_model.encode(item_all)

In [ ]:
prompt = f"""아래 광고 메세지의 구조 분석해라.분석의 목적은 메세지에서 구조화된 정보를 추출하기 위함이다.다양한 구조의 다른 메세지들도 분석할 예정인데, 이를 위한 prompt에 추가할 JSON이 아래와 같은 데, 개선점을 파악해서, 새로운 JSON을 추천해라.===광고 메세지 예시==={mms_pdf['msg'][:100]}===PROMPT에 넣을 JSON==={schema_cld}"""cot_list = []for row in few_shot_pdf.to_dict('records'):    prompt = f"""    당신은 캠페인 메시지에서 정보를 추출하는 전문가입니다. 다음과 같은 schema에 매칭되는 JSON 개체를 만들기 위한 Chain-of-Thought를 만들어주세요.    당신이 제공한 Chain-of-Thought는 다른 LLM이 유사한 광고 메세지에서 구조화된 정보를 추출하는 데 사용됩니다.    다른 LLM이 이 Chain-of-Thought를 사용하여 광고 메세지에서 구조화된 정보를 추출할 수 있도록, 예시 광고 메세지의 특징을 파악해야 하지만, 그렇다고 너무 의존적이어서는 안됩니다.     왜나허면, 동일한 메세지 분석에 사용되는 것이 아니기 때문입니다.    Chain-of-Thought는 한국어로 작성하고, 다른 LLM에 prompt로 바로 사용될 수 있도록 hain-of-Thought 결과만을 제공해 주세요.        ===광고 메세지 예시===    메세지 제목: {row['msg_head']}    메세지 내용: {row['msg_body']}    ===결과 Schema===    {schema_cld}    """    row['cot_chat'] = (llm_chat.invoke(prompt).content)    cot_list.append(row)    if len(cot_list) % 10 == 0:        print(f"Processed {len(cot_list)} rows")    # print((llm_chat.invoke(prompt).content))

In [ ]:
from langchain_anthropic import ChatAnthropicfrom langchain_core.prompts import ChatPromptTemplatefrom langchain_core.output_parsers import JsonOutputParserimport jsonimport re# Define your schemaschema_cld = {'properties': {    'message_info': {        'type': 'object',         'properties': {            'title': {'type': 'string', 'description': '광고 제목 - 원본 그대로 추출'},            'main_theme': {'type': 'string', 'description': '광고의 핵심 주제와 가치 제안을 명확하게 설명'},            'period': {'type': 'string', 'description': '이벤트/프로모션 기간 - 명시적으로 언급된 경우에만 추출, 없으면 "상시"로 설정'}        }    },    'purpose': {        'type': 'array',         'description': '광고의 주요 목적을 다음 중에서 선택(복수 가능): [상품 가입 유도, 대리점 방문 유도, 웹/앱 접속 유도, 이벤트 응모 유도, 혜택 안내, 쿠폰 제공 안내, 경품 제공 안내, 기타 정보 제공]'    },    'target': {        'type': 'array',         'items': {            'type': 'object',             'properties': {                'segment': {'type': 'string', 'description': '타겟 고객층 - 광고에서 명시적으로 언급하거나 암시한 대상 고객'},                'characteristics': {'type': 'string', 'description': '해당 타겟의 특성과 니즈'},                'priority': {'type': 'integer', 'description': '광고 내용에서의 중요도에 따른 타겟팅 우선순위 (1이 최우선)'}            }        }    },    'product': {        'type': 'array',        'items': {            'type': 'object',            'properties': {            'name': {'type': 'string', 'description': '광고하는 제품이나 서비스 이름 - "상품 후보 정보" 목록에서 일치하는 항목 선택'},            'id': {'type': 'string', 'description': '제품/서비스 ID - "상품 후보 정보"에서 name에 일치하는 항목의 ID 선택'},            'category': {'type': 'string', 'description': '제품/서비스 카테고리 - "상품 후보 정보"에서 확인'},            'benefit': {                'type': 'array',                'items': {                'type': 'object',                'properties': {                    'name': {'type': 'string', 'description': '제공되는 구체적인 혜택 내용 상세 설명'},                    'type': {'type': 'string', 'description': '혜택 유형: [할인, 쿠폰, 무료체험, 포인트, 사은품, 기타] 중에서 선택'},                    'target_audience': {'type': 'string', 'description': '혜택이 제공되는 특정 대상 (예: 만 13~34세, T멤버십 고객 등)'},                    'valid_period': {'type': 'string', 'description': '혜택 유효 기간 (있는 경우)'}                }                }            },            'conditions': {                'type': 'array',                'items': {                'type': 'object',                'properties': {                    'description': {'type': 'string', 'description': '혜택/구매를 받기 위한 구체적인 조건'},                    'type': {'type': 'string', 'description': '조건 유형: [약정기간, 요금제가입, 결합상품, 선착순, 카드결제, 기타] 중에서 선택'}                }                }            },            'related_products': {                'type': 'array',                'items': {                'type': 'object',                'properties': {                    'name': {'type': 'string', 'description': '주요 상품과 연계된 관련 상품/서비스 이름'},                    'relationship_type': {'type': 'string', 'description': '관계 유형: [결합상품, 부가서비스, 제휴서비스, 앱서비스, 기타] 중에서 선택'}                }                }            },            'action': {'type': 'string', 'description': '고객에게 기대하는 행동: [구매, 가입, 사용, 방문, 참여, 코드입력, 쿠폰다운로드, 기타] 중에서 선택'}            }        }    },    'channel': {        'type': 'array',         'items': {            'type': 'object',             'properties': {                'type': {'type': 'string', 'description': '채널 종류: [URL, 전화번호, 앱, 대리점] 중에서 선택'},                'value': {'type': 'string', 'description': '실제 URL, 전화번호, 앱 이름, 대리점 이름 등 구체적 정보'},                'action': {'type': 'string', 'description': '채널 목적: [가입, 추가 정보, 문의, 수신, 수신 거부] 중에서 선택'},                'benefit': {'type': 'string', 'description': '해당 채널 이용 시 특별 혜택'},                'store_code': {'type': 'string', 'description': "매장 코드 - tworldfriends.co.kr URL에서 D+숫자 9자리(D[0-9]{9}) 패턴의 코드 추출하여 대리점 채널에 설정"}            }        }    },    'metadata': {        'type': 'object',         'properties': {            'message_type': {'type': 'string', 'description': '메시지 유형 - 항상 "광고"로 설정'},            'target_response': {'type': 'string', 'description': '광고주가 고객으로부터 기대하는 반응 요약'},            'success_metrics': {'type': 'array', 'description': '이 캠페인의 성공을 측정하기 위한 지표들 추출'}        }    }}, 'required': ['purpose', 'target', 'product', 'channel', 'metadata'], 'objectType': 'object'}# Improved extraction guidanceextraction_guide = """### 분석 주요 포인트 ###1. 전체 메시지 분석: 먼저 광고 메시지를 전체적으로 읽고 주요 내용, 목적, 혜택, 타겟을 종합적으로 파악하세요.2. 상품 식별 및 관계 분석:    - "상품 후보 정보" 목록을 확인하여 광고에 언급된 제품/서비스와 일치하는 항목을 모두 선택하세요.   - 광고에 여러 상품이 포함된 경우 각 상품별로 별도 객체를 생성하고 related_products에 관계를 명시하세요.3. 혜택 및 조건 분석:    - 각 상품의 다양한 혜택을 benefit 배열에 추가하고 대상 고객층과 유효기간을 명시하세요.   - 각 제품별 혜택과 조건을 명확하게 구분하여 추출하고, 혜택을 받기 위한 조건들을 conditions 배열에 추가하세요.4. 채널 정보 분석:    - 모든 연락 채널(URL, 전화번호, 대리점 등)을 channel 배열에 빠짐없이 추가하세요.   - tworldfriends.co.kr URL이 있는 경우 반드시 매장 코드(D+숫자 패턴)를 추출하여 store_code에 포함하세요.5. 타겟 고객층 분석: 명시적으로 언급된 타겟 고객층을 target 배열에 우선순위대로 추가하세요.6. 정확한 정보 우선:    - 각 필드에 적합한 값을 선택할 때 광고 내용에 명시적으로 언급된 정보를 우선하고, 없는 경우에만 적절히 추론하세요.   - 결과 JSON은 완전하고 정확해야 하며, schema에 정의된 모든 필수 필드를 포함해야 합니다.7. 학습용 예시를 참고 정보로 사용하지 마세요.### 응답 형식 ###순수 JSON만 반환하세요. 설명이나 추가 텍스트는 포함하지 마세요."""# Custom parser to extract JSON from text responseclass CustomJsonOutputParser(JsonOutputParser):    def parse(self, text):        # Try to find JSON in the response using regex        json_match = re.search(r'({[\s\S]*})', text)        if json_match:            json_str = json_match.group(1)            try:                return json.loads(json_str)            except json.JSONDecodeError:                raise OutputParserException(f"Failed to parse JSON: {json_str}")        else:            raise OutputParserException(f"Could not find JSON in response: {text}")# Create a prompt template with improved system message - emphasize JSON-only outputprompt = ChatPromptTemplate.from_messages([    ("system", """당신은 SKT 캠페인 메시지에서 정확한 정보를 추출하는 전문가입니다. 아래 schema에 따라 광고 메시지를 분석하여 완전하고 정확한 JSON 객체를 생성해 주세요:{schema}{extraction_guide}{rag_context}### 학습용 예시 ###{few_shot_exm_str}이제 다음 광고 메시지를 분석하여 schema에 맞는 완전한 JSON 객체를 생성해 주세요.중요: 응답은 설명이나 추가 텍스트 없이 순수한 JSON 형식으로만 제공하세요. 응답은 '{{'로 시작하고 '}}'로 끝나야 합니다."""),    ("human", "{text}")])# Create the custom parserparser = CustomJsonOutputParser()# Create the chainextraction_chain = prompt | llm_cld37 | parser# Wrapper function to handle parsing failuresdef extract_json_from_response(response_text):    """Extract JSON from LLM response text even if it has explanatory text"""    try:        # First try to parse as pure JSON        return json.loads(response_text)    except json.JSONDecodeError:        # If that fails, try to extract JSON using regex        json_match = re.search(r'({[\s\S]*})', response_text)        if json_match:            json_str = json_match.group(1)            try:                return json.loads(json_str)            except json.JSONDecodeError:                print(f"Failed to parse extracted JSON pattern: {json_str}")                return None        else:            print(f"Could not find JSON pattern in response: {response_text}")            return None# Run the extraction with error handlingmsg_text = """### 분석할 메시지 ###광고 제목:[SK텔레콤] 2월 0 day 혜택 안내광고 내용:(광고)[SKT] 2월 0 day 혜택 안내__[2월 10일(토) 혜택]_만 13~34세 고객이라면_베어유 모든 강의 14일 무료 수강 쿠폰 드립니다!_(선착순 3만 명 증정)_▶ 자세히 보기: http://t-mms.kr/t.do?m=#61&s=24589&a=&u=https://bit.ly/3SfBjjc__■ 에이닷 X T 멤버십 시크릿코드 이벤트_에이닷 T 멤버십 쿠폰함에 ‘에이닷이빵쏜닷’을 입력해보세요!_뚜레쥬르 데일리우유식빵 무료 쿠폰을 드립니다._▶ 시크릿코드 입력하러 가기: https://bit.ly/3HCUhLM__■ 문의: SKT 고객센터(1558, 무료)_무료 수신거부 1504"""# Your existing code for similarities calculation and examples...similarities = cosine_similarity(            tfidf_desc.transform([msg_text]),            tfidf_matrix_desc        )[0]        # Assign similarities to dataframefew_shot_temp = few_shot_pdf.copy()  # Create a copy to avoid modifying originalfew_shot_temp['sim'] = similaritiesfew_shot_temp['rank'] = few_shot_temp["sim"].rank(method='min', ascending=False)# Get examples with better structure - make sure each example shows JSON only in the resultfew_shot_exm = []top_examples = few_shot_temp[    (few_shot_temp['rank'] >= 1) &     (few_shot_temp['rank'] <= 2)].sort_values("rank")# fs_model = "gpt"for _, r in top_examples.iterrows():    # Ensure the examples show proper JSON-only outputs    example_json = r[f'res_{fs_model}']        # If the example contains explanatory text, try to extract just the JSON    if not (example_json.startswith('{') and example_json.endswith('}')):        json_match = re.search(r'({[\s\S]*})', example_json)        if json_match:            example_json = json_match.group(1)        few_shot_exm.append(f"""[학습용 광고 메세지_{_}]\t광고 제목:{r['msg_head']}\t광고 내용:{r['msg_body']}[학습용 정답 결과_{_}]\t{example_json}""")few_shot_exm_str = "\n".join(few_shot_exm)# Enhanced entity matchingmatches = find_entities_in_text(    msg_text,     entity_list,     min_similarity=65,    ngram_size=3,    min_entity_length=2)product_info = ", ".join(set([match['text'] for match in matches]))stop_item_names = ['SK텔레콤','혜택안내','수신거부','멤버십','150','고객센터']mdf = pd.DataFrame(matches).query("text.str.replace(' ', '') not in @stop_item_names")mdf['item_id'] = mdf['data'].apply(lambda x: x['item_id'])mdf['category'] = mdf['data'].apply(lambda x: x['category'])product_info = json.dumps(mdf.rename(columns={'text':'item_name_in_message','matched_entity':'item_name_in_voca'})[['item_name_in_message','item_name_in_voca','item_id','category']].drop_duplicates().to_dict(orient='records'), ensure_ascii=False)rag_context = f"### 상품 정보 매핑 ###\n{product_info}"# Include extraction guide in promptprompt_text = prompt.format(    text=msg_text,     schema=str(schema_cld),     rag_context=rag_context,     few_shot_exm_str=few_shot_exm_str,    extraction_guide=extraction_guide)print("Prompt being sent to LLM:")print(prompt_text)print("\n" + "="*50 + "\n")try:    # Use the extraction chain with the custom parser    result_cld = extraction_chain.invoke({        "text": msg_text,         "schema": str(schema_cld),         "rag_context": rag_context,         "few_shot_exm_str": few_shot_exm_str,        "extraction_guide": extraction_guide    })    print(json.dumps(result_cld, indent=4, ensure_ascii=False))except Exception as e:    print(f"Error with extraction chain: {e}")        # Get direct response from LLM without parser    raw_response = llm_cld35.invoke(prompt_text)        # Extract JSON from the raw response    result_cld = extract_json_from_response(raw_response)    if result_cld:        print(json.dumps(result_cld, indent=4, ensure_ascii=False))    else:        print("Failed to extract valid JSON from the response")

In [ ]:
from langchain_anthropic import ChatAnthropicfrom langchain_core.prompts import ChatPromptTemplatefrom langchain_core.output_parsers import JsonOutputParserimport jsonimport re# Define your schemaschema_prd = {  'properties': {    'purpose': {        'type': 'array',         'description': '광고의 주요 목적을 다음 중에서 선택(복수 가능): [상품 가입 유도, 대리점 방문 유도, 웹/앱 접속 유도, 이벤트 응모 유도, 혜택 안내, 쿠폰 제공 안내, 경품 제공 안내, 기타 정보 제공]'    },    'product': {      'type': 'array',      'items': {        'type': 'object',        'properties': {          'name': {'type': 'string', 'description': '광고하는 제품이나 서비스 이름 - "상품 후보 정보" 목록에서 일치하는 항목 모두 선택'},          'id': {'type': 'string', 'description': '제품/서비스 ID - "상품 후보 정보"에서 name에 일치하는 항목의 ID 선택'},          'category': {'type': 'string', 'description': '제품/서비스 카테고리 - "상품 후보 정보"에서 확인'},          'benefit': {            'type': 'array',            'items': {              'type': 'object',              'properties': {                'name': {'type': 'string', 'description': '제공되는 구체적인 혜택 내용 상세 설명'},                'type': {'type': 'string', 'description': '혜택 유형: [할인, 쿠폰, 경품, 포인트, 무료체험, 사은품, 기타] 중에서 선택'},                'target_audience': {'type': 'string', 'description': '혜택이 제공되는 특정 대상 (예: 만 13~34세, T멤버십 고객, 장기 우수 고객 등)'},                'valid_period': {'type': 'string', 'description': '혜택 유효 기간이나 이벤트 기간 (있는 경우)'}              }            }          },          'conditions': {            'type': 'array',            'items': {              'type': 'object',              'properties': {                'description': {'type': 'string', 'description': '혜택/구매를 받기 위한 구체적인 조건'},                'type': {'type': 'string', 'description': '조건 유형: [약정기간, 요금제가입, 결합상품, 선착순, 카드결제, 기타] 중에서 선택'}              }            }          },          'related_products': {            'type': 'array',            'items': {              'type': 'object',              'properties': {                'name': {'type': 'string', 'description': '주요 상품과 연계된 관련 상품/서비스 이름'},                'id': {'type': 'string', 'description': '제품/서비스 ID - "상품 후보 정보"에서 name에 일치하는 항목의 ID 선택'},                'relationship_type': {'type': 'string', 'description': '관계 유형: [결합상품, 부가서비스, 제휴서비스, 앱서비스, 기타] 중에서 선택'}              }            }          },          'action': {'type': 'string', 'description': '고객에게 기대하는 행동: [구매, 가입, 사용, 방문, 참여, 코드입력, 쿠폰다운로드, 기타] 중에서 선택'}        }      }    }  },  'objectType': 'object'}# Improved extraction guidanceextraction_guide = """### 분석 시 고려사항 ###1. 하나의 광고에 여러 상품이 포함될 수 있으며, 각 상품별로 별도 객체 생성2. 특정 상품에 다양한 혜택이 제공될 수 있으며, 각 혜택은 별도 benefit 항목으로 추출3. 서로 다른 상품 간 관계가 있을 경우(결합상품, 앱서비스 등) related_products에 명시4. 혜택을 받기 위한 다양한 조건들을 모두 conditions 배열에 포함5. 대상 고객층이 특정된 경우(연령, 멤버십 등급 등) target_audience에 명시6. 시기별 프로모션(명절, 새학기 등) 특성을 valid_period에 포함### 분석 목표 ###"상품 후보 정보" 목록을 확인하여 광고에 언급된 제품/서비스와 일치하는 항목을 모두 선택하세요.### JSON 응답 형식 ###응답은 설명 없이 순수한 JSON 형식으로만 제공하세요. 응답의 시작과 끝은 '{'와 '}'여야 합니다. 어떠한 추가 텍스트나 설명도 포함하지 마세요."""# Custom parser to extract JSON from text responseclass CustomJsonOutputParser(JsonOutputParser):    def parse(self, text):        # Try to find JSON in the response using regex        json_match = re.search(r'({[\s\S]*})', text)        if json_match:            json_str = json_match.group(1)            try:                return json.loads(json_str)            except json.JSONDecodeError:                raise OutputParserException(f"Failed to parse JSON: {json_str}")        else:            raise OutputParserException(f"Could not find JSON in response: {text}")# Create a prompt template with improved system message - emphasize JSON-only outputprompt = ChatPromptTemplate.from_messages([    ("system", """당신은 SKT 캠페인 메시지에서 정확한 정보를 추출하는 전문가입니다. 아래 schema에 따라 광고 메시지를 분석하여 완전하고 정확한 JSON 객체를 생성해 주세요:{schema}{extraction_guide}=====참고 정보====={rag_context}이제 다음 광고 메시지를 분석하여 schema에 맞는 완전한 JSON 객체를 생성해 주세요.중요: 응답은 설명이나 추가 텍스트 없이 순수한 JSON 형식으로만 제공하세요. 응답은 '{{'로 시작하고 '}}'로 끝나야 합니다."""),    ("human", "{text}")])# Create the custom parserparser = CustomJsonOutputParser()# Create the chainextraction_chain = prompt | llm_cld37 | parser# Wrapper function to handle parsing failuresdef extract_json_from_response(response_text):    """Extract JSON from LLM response text even if it has explanatory text"""    try:        # First try to parse as pure JSON        return json.loads(response_text)    except json.JSONDecodeError:        # If that fails, try to extract JSON using regex        json_match = re.search(r'({[\s\S]*})', response_text)        if json_match:            json_str = json_match.group(1)            try:                return json.loads(json_str)            except json.JSONDecodeError:                print(f"Failed to parse extracted JSON pattern: {json_str}")                return None        else:            print(f"Could not find JSON pattern in response: {response_text}")            return None# Run the extraction with error handlingmsg_text = """광고 제목:[SK텔레콤] 2월 0 day 혜택 안내	광고 내용:(광고)[SKT] 2월 0 day 혜택 안내__[2월 10일(토) 혜택]_만 13~34세 고객이라면_베어유 모든 강의 14일 무료 수강 쿠폰 드립니다!_(선착순 3만 명 증정)_▶ 자세히 보기: http://t-mms.kr/t.do?m=#61&s=24589&a=&u=https://bit.ly/3SfBjjc__■ 에이닷 X T 멤버십 시크릿코드 이벤트_에이닷 T 멤버십 쿠폰함에 ‘에이닷이빵쏜닷’을 입력해보세요!_뚜레쥬르 데일리우유식빵 무료 쿠폰을 드립니다._▶ 시크릿코드 입력하러 가기: https://bit.ly/3HCUhLM__■ 문의: SKT 고객센터(1558, 무료)_무료 수신거부 1504 """msg_text = """'\n  \t광고 제목:통화 부가서비스를 패키지로 저렴하게!\n  \t광고 내용:(광고)[SKT] 콜링플러스 이용 안내  #04 고객님, 안녕하세요. <콜링플러스>에 가입하고 콜키퍼, 컬러링, 통화가능통보플러스까지 총 3가지의 부가서비스를 패키지로 저렴하게 이용해보세요.  ■ 콜링플러스 - 이용요금: 월 1,650원, 부가세 포함 - 콜키퍼(550원), 컬러링(990원), 통화가능통보플러스(770원)를 저렴하게 이용할 수 있는 상품  ■ 콜링플러스 가입 방법 - T월드 앱: 오른쪽 위에 있는 돋보기를 눌러 콜링플러스 검색 > 가입  ▶ 콜링플러스 가입하기: http://t-mms.kr/t.do?m=#61&u=https://skt.sh/17tNH  ■ 유의 사항 - 콜링플러스에 가입하면 기존에 이용 중인 콜키퍼, 컬러링, 통화가능통보플러스 서비스는 자동으로 해지됩니다. - 기존에 구매한 컬러링 음원은 콜링플러스 가입 후에도 계속 이용할 수 있습니다.(시간대, 발신자별 설정 정보는 다시 설정해야 합니다.)  * 최근 다운로드한 음원은 보관함에서 무료로 재설정 가능(다운로드한 날로부터 1년 이내)   ■ 문의: SKT 고객센터(114)  SKT와 함께해주셔서 감사합니다.  무료 수신거부 1504\n    ', """# Your existing code for similarities calculation and examples...similarities = cosine_similarity(            tfidf_desc.transform([msg_text]),            tfidf_matrix_desc        )[0]        # Assign similarities to dataframefew_shot_temp = few_shot_pdf.copy()  # Create a copy to avoid modifying originalfew_shot_temp['sim'] = similaritiesfew_shot_temp['rank'] = few_shot_temp["sim"].rank(method='min', ascending=False)# Get examples with better structure - make sure each example shows JSON only in the resultfew_shot_exm = []top_examples = few_shot_temp[    (few_shot_temp['rank'] >= 1) &     (few_shot_temp['rank'] <= 2)].sort_values("rank")# fs_model = "gpt"for _, r in top_examples.iterrows():    # Ensure the examples show proper JSON-only outputs    example_json = r[f'res_{fs_model}']        # If the example contains explanatory text, try to extract just the JSON    if not (example_json.startswith('{') and example_json.endswith('}')):        json_match = re.search(r'({[\s\S]*})', example_json)        if json_match:            example_json = json_match.group(1)    example_json = {'product': extract_json_objects(example_json)[0]['product']}    example_json = str(example_json)        few_shot_exm.append(f"""[학습용 광고 메세지_{_}]\t광고 제목:{r['msg_head']}\t광고 내용:{r['msg_body']}[학습용 정답 결과_{_}]\t{example_json}""")few_shot_exm_str = "\n".join(few_shot_exm)# Enhanced entity matchingmatches = find_entities_in_text(    msg_text,     entity_list,     min_similarity=50,    high_score_threshold=50,    overlap_tolerance=0.5)product_info = ", ".join(set([match['text'] for match in matches]))stop_item_names = ['SK텔레콤','혜택안내','수신거부','멤버십','150','고객센터']mdf = pd.DataFrame(matches).query("text.str.replace(' ', '') not in @stop_item_names")mdf['item_id'] = mdf['data'].apply(lambda x: x['item_id'])mdf['category'] = mdf['data'].apply(lambda x: x['category'])product_info = json.dumps(mdf.rename(columns={'text':'item_name_in_message','matched_entity':'item_name_in_voca'})[['item_name_in_message','item_name_in_voca','item_id','category']].drop_duplicates().to_dict(orient='records'), ensure_ascii=False)rag_context = f"\t상품 후보 정보: {product_info}"# Include extraction guide in promptprompt_text = prompt.format(    text=msg_text,     schema=str(schema_prd),     rag_context=rag_context,     extraction_guide=extraction_guide)print("Prompt being sent to LLM:")print(prompt_text)print("\n" + "="*50 + "\n")try:    # Use the extraction chain with the custom parser    result_cld = extraction_chain.invoke({        "text": msg_text,         "schema": str(schema_prd),         "rag_context": rag_context,         "extraction_guide": extraction_guide    })    print(json.dumps(result_cld, indent=4, ensure_ascii=False))except Exception as e:    print(f"Error with extraction chain: {e}")        # Get direct response from LLM without parser    raw_response = llm_cld35.invoke(prompt_text)        # Extract JSON from the raw response    result_cld = extract_json_from_response(raw_response)    if result_cld:        print(json.dumps(result_cld, indent=4, ensure_ascii=False))    else:        print("Failed to extract valid JSON from the response")

In [ ]:
import jsonimport reimport pandas as pdfrom openai import OpenAIfrom sklearn.feature_extraction.text import TfidfVectorizerfrom sklearn.metrics.pairwise import cosine_similarity# Define your schemaschema_cld_prd_graph = {    "properties": {      'purpose': {        'type': 'array',         'description': '광고의 주요 목적을 다음 중에서 선택(복수 가능): [상품 가입 유도, 대리점 방문 유도, 웹/앱 접속 유도, 이벤트 응모 유도, 혜택 안내, 쿠폰 제공 안내, 경품 제공 안내, 기타 정보 제공]'      },      "product": {        "type": "array",        "description": "메시지에서 추출한 상품 간의 관계 정보입니다.",        "items": {          "type": "object",          "description": "하나의 상품과 다른 상품 간의 관계를 나타냅니다.",          "properties": {            "source_item": {              "type": "object",              "description": "관계의 주체가 되는 상품입니다. 보통 혜택이나 서비스를 제공하는 주체입니다.",              "properties": {                "id": {                  "type": "string",                  "description": "상품의 고유 식별자(ID)입니다."                },                "name_in_voca": {                  "type": "string",                  "description": "아래 상품 후보 정보에 정의된 상품 이름입니다."                },                "name_in_message": {                  "type": "string",                  "description": "메시지 텍스트 내에서 사용된 상품 이름입니다."                },                "description": {                  "type": "string",                  "description": "상품에 대한 간략한 설명입니다."                }              },              "required": ["id", "name_in_voca", "name_in_message"]            },            "relation": {              "type": "object",              "description": "두 상품 간의 관계를 설명하는 객체입니다.",              "properties": {                "type": {                  "type": "string",                  "description": "관계의 유형입니다. '제공', '협업', '활용', '포함', '결합', '부가', '제휴' 등의 값을 가질 수 있습니다."                },                "description": {                  "type": "string",                  "description": "관계에 대한 상세 설명입니다."                }              },              "required": ["type"]            },            "target_item": {              "type": "object",              "description": "관계의 대상이 되는 상품입니다. 보통 혜택이나 서비스를 받는 대상입니다.",              "properties": {                "id": {                  "type": "string",                  "description": "상품의 고유 식별자(ID)입니다."                },                "name_in_voca": {                  "type": "string",                  "description": "아래 상품 후보 정보에 정의된 상품 이름입니다."                },                "name_in_message": {                  "type": "string",                  "description": "메시지 텍스트 내에서 사용된 상품 이름입니다."                },                "description": {                  "type": "string",                  "description": "상품에 대한 간략한 설명입니다."                }              },              "required": ["id", "name_in_voca", "name_in_message"]            }          },          "required": ["source_item", "relation", "target_item"]        },        "examples": {          "relation_types": {            "제공": "한 상품이 다른 상품에 혜택을 제공하는 관계입니다. 예: 0 day가 베어유 무료 수강 쿠폰을 제공",            "협업": "두 상품이 공동으로 이벤트나 프로모션을 진행하는 관계입니다. 예: 에이닷과 T 멤버십이 시크릿코드 이벤트 진행",            "활용": "한 상품이 다른 상품의 기능을 활용하는 관계입니다. 예: 에이닷이 T 멤버십 쿠폰함 기능을 활용",            "포함": "한 상품이 다른 상품을 포함하는 관계입니다. 예: T 우주 패스가 YouTube Premium을 포함",            "결합": "두 개 이상의 상품이 하나로 결합되어 새로운 가치를 창출하는 관계입니다. 예: 인터넷과 TV가 결합하여 할인 혜택 제공",            "부가": "기본 상품에 추가적인 상품이나 서비스가 부가되는 관계입니다. 예: SK텔레콤에 T 올케어플러스5가 부가서비스로 추가",            "제휴": "서로 다른 회사나 브랜드의 상품이 제휴 관계를 맺고 협력하는 관계입니다. 예: SK텔레콤과 Uber Taxi의 제휴"          }        }      }    },    "required": ["purpose","product"]}# Improved extraction guidance with clear focus on JSON outputextraction_guide = """### 분석 가이드 ###다음과 같은 너무 일반적인 명칭의 개체명은 제외하세요: 인터넷, 이벤트, 휴대폰, 친구, 할인, 적립, 전화 등순수 JSON만 반환하세요. 설명이나 추가 텍스트는 포함하지 마세요."""# Custom parser to extract JSON from text responseclass CustomJsonOutputParser(JsonOutputParser):    def parse(self, text):        # Try to find JSON in the response using regex        json_match = re.search(r'({[\s\S]*})', text)        if json_match:            json_str = json_match.group(1)            try:                return json.loads(json_str)            except json.JSONDecodeError:                raise OutputParserException(f"Failed to parse JSON: {json_str}")        else:            raise OutputParserException(f"Could not find JSON in response: {text}")# Create a prompt template with improved system message - emphasize JSON-only outputprompt = ChatPromptTemplate.from_messages([    ("system", """당신은 SKT 캠페인 메시지에서 정확한 정보를 추출하는 전문가입니다. 아래 schema에 따라 광고 메시지를 분석하여 완전하고 정확한 JSON 객체를 생성해 주세요:{schema}{extraction_guide}=====참고 정보====={rag_context}이제 다음 광고 메시지를 분석하여 schema에 맞는 완전한 JSON 객체를 생성해 주세요.중요: 응답은 설명이나 추가 텍스트 없이 순수한 JSON 형식으로만 제공하세요. 응답은 '{{'로 시작하고 '}}'로 끝나야 합니다."""),    ("human", "{text}")])# Create the custom parserparser = CustomJsonOutputParser()# Create the chainextraction_chain = prompt | llm_cld37 | parser# Wrapper function to handle parsing failuresdef extract_json_from_response(response_text):    """Extract JSON from LLM response text even if it has explanatory text"""    try:        # First try to parse as pure JSON        return json.loads(response_text)    except json.JSONDecodeError:        # If that fails, try to extract JSON using regex        json_match = re.search(r'({[\s\S]*})', response_text)        if json_match:            json_str = json_match.group(1)            try:                return json.loads(json_str)            except json.JSONDecodeError:                print(f"Failed to parse extracted JSON pattern: {json_str}")                return None        else:            print(f"Could not find JSON pattern in response: {response_text}")            return None# Enhanced entity matching - This assumes you have the find_entities_in_text and entity_list functions definedentity_list_trunc = [d for d in entity_list if d[1]['create_dt']>20200101]stop_item_names = ['SK텔레콤','혜택안내','수신거부','멤버십','150','고객센터','TBD Product','SKT','인터넷','이벤트','휴대폰','친구','할인','적립','전화','product','구독상품','mms','가입']pred_results = []for m in mms_pdf.query("not msg.str.contains('대리점')").sample(n=10).to_dict(orient='records'):    # Run the extraction with pre-processing    msg_text = f"""  \t광고 제목:{m['msg_nm']}  \t광고 내용:{m['mms_phrs']}    """#     msg_text = """# '\n  \t광고 제목:통화 부가서비스를 패키지로 저렴하게!\n  \t광고 내용:(광고)[SKT] 콜링플러스 이용 안내  #04 고객님, 안녕하세요. <콜링플러스>에 가입하고 콜키퍼, 컬러링, 통화가능통보플러스까지 총 3가지의 부가서비스를 패키지로 저렴하게 이용해보세요.  ■ 콜링플러스 - 이용요금: 월 1,650원, 부가세 포함 - 콜키퍼(550원), 컬러링(990원), 통화가능통보플러스(770원)를 저렴하게 이용할 수 있는 상품  ■ 콜링플러스 가입 방법 - T월드 앱: 오른쪽 위에 있는 돋보기를 눌러 콜링플러스 검색 > 가입  ▶ 콜링플러스 가입하기: http://t-mms.kr/t.do?m=#61&u=https://skt.sh/17tNH  ■ 유의 사항 - 콜링플러스에 가입하면 기존에 이용 중인 콜키퍼, 컬러링, 통화가능통보플러스 서비스는 자동으로 해지됩니다. - 기존에 구매한 컬러링 음원은 콜링플러스 가입 후에도 계속 이용할 수 있습니다.(시간대, 발신자별 설정 정보는 다시 설정해야 합니다.)  * 최근 다운로드한 음원은 보관함에서 무료로 재설정 가능(다운로드한 날로부터 1년 이내)   ■ 문의: SKT 고객센터(114)  SKT와 함께해주셔서 감사합니다.  무료 수신거부 1504\n    ', # """    # Enhanced entity matching    matches = find_entities_in_text(        replace_strings(msg_text, {x:'#' for x in stop_item_names}),         entity_list_trunc,         min_similarity=50,        high_score_threshold=50,        overlap_tolerance=0.5    )    matches = [t for t in matches if not t['text'] in stop_item_names]    if len(matches)<1:continue    product_info = ", ".join(set([match['text'] for match in matches]))        mdf = pd.DataFrame(matches).query("text.str.replace(' ', '') not in @stop_item_names")    mdf['item_id'] = mdf['data'].apply(lambda x: x['item_id'])    mdf['category'] = mdf['data'].apply(lambda x: x['category'])    product_info = json.dumps(mdf.rename(columns={'text':'item_name_in_message','matched_entity':'item_name_in_voca'})[['item_name_in_message','item_name_in_voca','item_id','category']].drop_duplicates().to_dict(orient='records'), ensure_ascii=False)    rag_context = f"\t상품 후보 정보: {product_info}"    # Include extraction guide in prompt    prompt_text = prompt.format(        text=replace_strings(msg_text, {x:'#' for x in stop_item_names}),         schema=str(schema_cld_prd_graph),         rag_context=rag_context,         extraction_guide=extraction_guide     )    print("Prompt being sent to LLM:")    print(prompt_text)    print("\n" + "="*50 + "\n")    try:        # Use the extraction chain with the custom parser        result_cld = extraction_chain.invoke({            "text": msg_text,             "schema": str(schema_cld_prd_graph),             "rag_context": rag_context,             "extraction_guide": extraction_guide        })                # print(json.dumps(result_cld, indent=4, ensure_ascii=False))    except Exception as e:        print(f"Error with extraction chain: {e}")                # Get direct response from LLM without parser        raw_response = llm_cld35.invoke(prompt_text)                # Extract JSON from the raw response        result_cld = extract_json_from_response(raw_response)        # if result_cld:        #     print(json.dumps(result_cld, indent=4, ensure_ascii=False))        # else:        #     print("Failed to extract valid JSON from the response")          finally:        pred_results.append({"msg":msg_text, "rag":rag_context, "res":result_cld})    if len(pred_results)%5==0:      print(len(pred_results))    time.sleep(2.0)    # break

In [ ]:

# pd.DataFrame(pred_results).to_csv(f"./data/prod_graph.csv", index=False, mode='w')
# pd.read_csv("./data/prod_graph.csv").shape
# [item for sublist in list_of_lists for item in sublist]
# [p for p in data if p['source_item']['name_in_message'] in stop_item_names]

In [216]:
import networkx as nx
def build_graph_from_dict_list(dict_list):
    # Create a directed graph
    G = nx.DiGraph()
    # Keep track of nodes and edges we've added to avoid duplicates
    added_nodes = set()
    added_edges = set()
    for item in dict_list:
        source = item['source_item']
        target = item['target_item']
        relation = item['relation']
        # Create unique identifiers for nodes
        source_id = source['id']
        target_id = target['id']
        # Add nodes if they don't exist already
        if source_id not in added_nodes:
            G.add_node(source_id,
                       name_in_voca=source.get('name_in_voca', ''),
                       name_in_message=source.get('name_in_message', ''),
                       description=source.get('description', ''))
            added_nodes.add(source_id)
        if target_id not in added_nodes:
            G.add_node(target_id,
                       name_in_voca=target.get('name_in_voca', ''),
                       name_in_message=target.get('name_in_message', ''),
                       description=target.get('description', ''))
            added_nodes.add(target_id)
        # Create a unique edge identifier
        edge_key = (source_id, target_id, relation['type'])
        # Add edge if it doesn't exist already
        if edge_key not in added_edges:
            G.add_edge(source_id, target_id,
                      relation_type=relation['type'],
                      description=relation.get('description', ''))
            added_edges.add(edge_key)
    return G
# Your list of dictionaries
data = sum([d['res']['product'] for d in pred_results],[])
data = [p for p in data if not p['source_item']['name_in_message'] in stop_item_names]
# Build the graph
G = build_graph_from_dict_list(data)
# # Verify results
# print(f"Number of nodes: {G.number_of_nodes()}")
# print(f"Number of edges: {G.number_of_edges()}")
# print("\nNodes:")
# for node, attributes in G.nodes(data=True):
#     print(f"  {node}: {attributes}")
# print("\nEdges:")
# for source, target, attributes in G.edges(data=True):
#     print(f"  {source} -> {target}: {attributes}")

In [ ]:
import networkx as nx
import pandas as pd
def get_important_nodes(G, top_n=None, measures=None):
    """
    Calculate importance metrics for nodes in graph G.
    Parameters:
    - G: NetworkX graph
    - top_n: Number of top nodes to return (None for all)
    - measures: List of centrality measures to calculate
               (None for all basic measures)
    Returns:
    - DataFrame with nodes and their importance scores
    """
    if measures is None:
        measures = ['degree', 'in_degree', 'out_degree', 'betweenness',
                   'closeness', 'eigenvector', 'pagerank']
    # Initialize results dictionary
    results = {node: {'node_id': node} for node in G.nodes()}
    # Calculate selected centrality measures
    if 'degree' in measures:
        for node, degree in G.degree():
            results[node]['degree'] = degree
    if 'in_degree' in measures and isinstance(G, nx.DiGraph):
        for node, in_degree in G.in_degree():
            results[node]['in_degree'] = in_degree
    if 'out_degree' in measures and isinstance(G, nx.DiGraph):
        for node, out_degree in G.out_degree():
            results[node]['out_degree'] = out_degree
    if 'betweenness' in measures:
        betweenness = nx.betweenness_centrality(G)
        for node, score in betweenness.items():
            results[node]['betweenness'] = score
    if 'closeness' in measures:
        try:
            closeness = nx.closeness_centrality(G)
            for node, score in closeness.items():
                results[node]['closeness'] = score
        except nx.NetworkXError:
            # Closeness centrality is not defined for disconnected graphs
            pass
    if 'eigenvector' in measures:
        try:
            eigenvector = nx.eigenvector_centrality(G, max_iter=100)
            for node, score in eigenvector.items():
                results[node]['eigenvector'] = score
        except nx.PowerIterationFailedConvergence:
            # Eigenvector centrality might not converge
            pass
    if 'pagerank' in measures:
        pagerank = nx.pagerank(G)
        for node, score in pagerank.items():
            results[node]['pagerank'] = score
    # Convert to DataFrame and sort
    df = pd.DataFrame(list(results.values()))
    # Return top N nodes if specified
    if top_n is not None and top_n < len(df):
        # For each measure, get top_n nodes
        important_nodes = set()
        for measure in measures:
            if measure in df.columns:
                top_indices = df.nlargest(top_n, measure)['node_id'].values
                important_nodes.update(top_indices)
        # Filter dataframe to only include important nodes
        df = df[df['node_id'].isin(important_nodes)]
    return df
# Example usage
# G = build_graph_from_dict_list(data)
# Get all centrality measures
importance_df = get_important_nodes(G)
print("All centrality measures:")
print(importance_df)
# Get only top 2 nodes based on degree and pagerank
top_nodes = get_important_nodes(G, top_n=2, measures=['degree', 'pagerank'])
print("\nTop 2 nodes by degree and PageRank:")
print(top_nodes)
# Access node attributes for important nodes
print("\nDetails of important nodes:")
for node_id in top_nodes['node_id']:
    attributes = G.nodes[node_id]
    print(f"{node_id}: {attributes}")

In [ ]:
pred_results[0]

In [ ]:
mms_pdf.query("msg.str.contains('콜링플러스') and msg.str.contains('통화 부가서비스')")['msg_nm'].values[0]
mms_pdf.query("msg.str.contains('콜링플러스') and msg.str.contains('통화 부가서비스')")['mms_phrs'].values[0]


In [ ]:
msg = f"""광고 제목:{mms_pdf.query("msg.str.contains('콜링플러스') and msg.str.contains('통화 부가서비스')")['msg_nm'].values[0]}  	광고 내용:{mms_pdf.query("msg.str.contains('콜링플러스') and msg.str.contains('통화 부가서비스')")['mms_phrs'].values[0]}"""entity_list_test = [x for x in entity_list_trunc                #      if '우주' in x[0]                     ] #["T 우주패스", "우주패스", "T멤버십", "프리미엄패스"][x for x in find_entities_in_text(        replace_strings(msg, {x:'#' for x in stop_item_names}),         entity_list_test,         min_similarity=50,        high_score_threshold=50,        overlap_tolerance=0.5)]

In [92]:
similarities = cosine_similarity(
            tfidf_desc.transform([msg_text]),
            tfidf_matrix_desc
        )[0]
# Assign similarities to dataframe
few_shot_temp = few_shot_pdf.copy()  # Create a copy to avoid modifying original
few_shot_temp['sim'] = similarities
few_shot_temp['rank'] = few_shot_temp["sim"].rank(method='min', ascending=False)
# Get examples with better structure - make sure each example shows JSON only in the result
few_shot_exm = []
top_examples = few_shot_temp[
    (few_shot_temp['rank'] >= 2) &
    (few_shot_temp['rank'] <= 2)
].sort_values("rank")
for r in top_examples.to_dict("records"):
    msg = f"""
    광고 제목:{r['msg_head']}
    광고 내용:{r['msg_body']}
    """
    result_cld = extraction_chain.invoke({
        "text": msg,
        "schema": str(schema_prd),
        "rag_context": rag_context,
        "extraction_guide": extraction_guide
    })
    example_json = json.dumps(result_cld, indent=4, ensure_ascii=False)
    few_shot_exm.append(f"""
[학습용 광고 메세지_{_}]
\t* 광고 제목:{r['msg_head']}
\t* 광고 내용:{r['msg_body']}
[학습용 정답 결과_{_}]
\t{example_json}""")
few_shot_exm_str = ("\n".join(few_shot_exm))

In [ ]:
# Create the system message with clear JSON output requirements
system_message = f"""당신은 SKT 캠페인 메시지에서 정확한 정보를 추출하는 전문가입니다. 아래 schema에 따라 광고 메시지를 분석하여 완전하고 정확한 JSON 객체를 생성해 주세요:
{json.dumps(schema_prd, indent=2, ensure_ascii=False)}
{extraction_guide}
"""
# Create message content with structured sections
user_message = f"""### 분석 대상 광고 메세지 ###
{msg_text}
### 관련 참고 정보 ###
{rag_context}
### 학습용 예시 ###
{few_shot_exm_str}
"""
# print('%'*15+" System Message "+'%'*15+"\n\n")
# print(system_message)
print('%'*15+" User Message "+'%'*15+"\n\n")
print(user_message)
try:
    # Use OpenAI's ChatCompletion with the current API format
    response = client.chat.completions.create(
        model="skt/a.x-3-lg",  # Or your preferred OpenAI model
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        temperature=0.0,
        max_tokens=4000,
        top_p=0.95,  # Reduces randomness
        frequency_penalty=0.0,  # Avoid repetition in JSON
        presence_penalty=0.0,
        response_format={"type": "json_object"}  # Explicitly request JSON format
    )
    # Extract the JSON from the response
    result_json_text = response.choices[0].message.content
    json_objects = extract_json_objects(result_json_text)[0]
    print("\n\n\n"+'%'*15+" LLM Result "+'%'*15+"\n\n")
    print(json.dumps(json_objects, indent=4, ensure_ascii=False))
except Exception as e:
    print(f"Error with API call: {e}")

In [ ]:
mdf = pd.DataFrame(matches).query("text.str.replace(' ', '') not in @stop_item_names")
mdf['item_id'] = mdf['data'].apply(lambda x: x['item_id'])
mdf['category'] = mdf['data'].apply(lambda x: x['category'])
product_info = json.dumps(mdf.rename(columns={'text':'item_name_in_message','matched_entity':'item_name_in_voca'})[['item_name_in_message','item_name_in_voca','item_id','category']].drop_duplicates().to_dict(orient='records'), ensure_ascii=False)
product_info

In [ ]:
user_message = f"""다음은 광고 메시지에서 추출한 상품들의 정보입니다.각 상품별 광고 목적을 아래 분류 기준에 따라 추론하여, 상품 정보 JSON에 각 item별 "purpose" 태그로 추가해 주세요.### 광고 목적 분류 기준 ###1. 신규 가입 유도: 통신 서비스, 멤버십, 구독 서비스 등에 새롭게 가입하도록 유도2. 재가입/연장 유도: 만료된 서비스의 재가입이나 구독 연장을 유도3. 상품 구매: 휴대폰, 액세서리 등 물리적 제품의 구매를 유도4. 사전예약: 출시 예정 제품의 사전 예약을 유도5. 서비스 이용: 기존 서비스, 앱, 콘텐츠의 활성 사용을 장려6. 단기 프로모션 참여: 한정된 기간의 할인, 이벤트 등 참여 유도7. 매장 방문: 오프라인 매장 방문을 유도8. 브랜드 인지도 증진: 특정 제품이나 서비스보다 브랜드 자체의 인지도 향상이 목적9. 고객 충성도 강화: 기존 고객의 충성도를 높이기 위한 혜택 안내10. 교차판매/상향판매: 기존 고객에게 추가 상품이나 상위 서비스 판매 유도11. 정보 제공: 공지사항, 보안 경고 등 정보성 메시지12. 제외: 광고 대상 상품이 아니고, 단순히 광고 메시지 구성 요소인 경우각 광고는 주요 목적과 부가적 목적이 있을 수 있습니다. 가장 핵심적인 목적을 "purpose" 값으로 지정하되, 필요한 경우 "secondary_purpose"를추가하여 보조적인 목적도 표시해 주세요.### 상품 정보 ###{product_info}### 분석 대상 광고 메시지 ###{msg_text}분석 결과는 기존 JSON 구조를 유지하면서 각 항목에 "purpose"와 필요 시 "secondary_purpose" 필드를 추가해 주세요."""try:    # Use OpenAI's ChatCompletion with the current API format    response = client.chat.completions.create(        model="skt/a.x-3-lg",  # Or your preferred OpenAI model        messages=[            # {"role": "system", "content": system_message},            {"role": "user", "content": user_message}        ],        temperature=0.0,        max_tokens=4000,        top_p=0.95,  # Reduces randomness        frequency_penalty=0.0,  # Avoid repetition in JSON        presence_penalty=0.0,        # response_format={"type": "json_object"}  # Explicitly request JSON format    )        # Extract the JSON from the response    result_json_text = response.choices[0].message.content    json_objects = extract_json_objects(result_json_text)    print("\n\n\n"+'%'*15+" LLM Result "+'%'*15+"\n\n")    print(json.dumps(json_objects, indent=4, ensure_ascii=False))            except Exception as e:    print(f"Error with API call: {e}")# extraction_chain = prompt | llm_cld37 | parser# prompt = ChatPromptTemplate.from_messages([#     ("human", "{user_message}")# ])# extraction_chain = prompt | llm_cld37 | parser# result_cld = extraction_chain.invoke({#         "user_message": user_message, #     })# print(json.dumps(result_cld, indent=4, ensure_ascii=False))

In [ ]:
mdf = mms_pdf.query("offer_dt>='20240101'").rename(columns={'msg_nm':'head','mms_phrs':'body'})
print("\n\n".join([d['msg'] for d in mdf.sample(n=100).to_dict("records")]))

In [ ]:
for index in range(len(result_cld['product'])):
    item_name = result_cld['product'][index]['name'] #+ " " + result['광고 상품'][0]['상품 정보']
    # get_relevant_context(item_name, vectorstore, topk=20)['title']
    matches = find_entities_in_text(
        result_cld['product'][index]['name'],
        entity_list,
        min_similarity=70,  # Lower threshold for more matches
        ngram_size=3,       # Bigrams work well for Korean
        min_entity_length=3 # Only consider entities with 3+ characters
    )
    schema_res = {
        "ext_item_nm":{"type":"string","description":"기준 item의 name"},
        "item_id":{"type":"string","description":"가장 유사한 item의 id"},
        "item_nm":{"type":"string","description":"가장 유사한 item의 name"},
        "domain":{"type":"string","description":"해당 item의 도메인"},
        "reason":{"type":"string","description":"선택한 이유"}
    }
    prompt = f"""
    아래 광고 메세지에서 추출한 상품 정보와 후보 상품 리스트 중에서 광고 메세지의 맥락에 가장 부합하는 상품을 선택해라.  다음과 같은 schema에 참고하여 JSON 개체로 만들어 주세요:\n {schema_res}
    ---상품 정보---
    상품명: {result_cld['product'][index]['name']}
    카테고리: {result_cld['product'][index]['category']}
    혜택: {result_cld['product'][index]['benefit']}
    조건: {result_cld['product'][index]['conditions']}
    반응: {result_cld['product'][index]['action']}
    ---원본 광고 메세지---
    {msg_text}
    ---후보 상품 리스트---
    {pd.DataFrame(matches)}
    """
    res_cld = extract_json_objects(llm_cld37.invoke(prompt).content)[0]
    print(json.dumps(res_cld, indent=4, ensure_ascii=False))

In [ ]:
prompt = f"""
아래 광고 메세지의 구조 분석해라.
분석의 목적은 메세지에서 구조화된 정보를 추출하기 위함이다.
다양한 구조의 다른 메세지들도 분석할 예정인데, 이를 위한 prompt에 추가할 JSON이 아래와 같은 데, 개선점을 파악해서, 새로운 JSON을 추천해라.
===광고 메세지 예시===
{mdf[:15]}
===PROMPT에 넣을 JSON===
{schema_ax}
"""
response = client.chat.completions.create(
    model="skt/a.x-3-lg",  # or your specific model
    messages=[
        {"role": "user", "content":prompt}
    ],
    temperature=0.0
)
print(response.choices[0].message.content)


In [ ]:
import jsonimport reimport pandas as pdfrom openai import OpenAIfrom sklearn.feature_extraction.text import TfidfVectorizerfrom sklearn.metrics.pairwise import cosine_similarity# Initialize the OpenAI clientclient = OpenAI(    api_key = llm_api_key,    base_url = llm_api_url)def extract_store_code(text):    # Pattern to find store code in tworldfriends.co.kr URL    pattern1 = r'tworldfriends\.co\.kr/([D][0-9]{9})'    # Pattern to find store code in detail?code= parameter    pattern2 = r'code=([D][0-9\-]+)'        # Try first pattern    match = re.search(pattern1, text)    if match:        return match.group(1)        # Try second pattern    match = re.search(pattern2, text)    if match:        return match.group(1)        return Nonedef extract_phone_numbers(text):    # Patterns for various phone number formats in Korea    patterns = [        r'(\d{3,4}-\d{3,4}-\d{4})',  # Format: XXX-XXXX-XXXX or XXXX-XXXX-XXXX        r'(0\d{1,2}-\d{3,4}-\d{4})',  # Format: 0X-XXXX-XXXX or 0XX-XXXX-XXXX        r'(0\d{3}-\d{3,4}-\d{4})',    # Format: 0XXX-XXX-XXXX or 0XXX-XXXX-XXXX        r'(\d{4}-\d{4})',             # Format: XXXX-XXXX        r'(1\d{3})',                  # Short numbers like 1588        r'(0507-\d{4}-\d{4})'         # Format: 0507-XXXX-XXXX (virtual numbers)    ]        phone_numbers = []    for pattern in patterns:        matches = re.findall(pattern, text)        phone_numbers.extend(matches)        return phone_numbersdef extract_unsubscribe_info(text):    # Look for unsubscribe text and associated number    unsubscribe_pattern = r'수신\s*거부\s*(\d+)'    match = re.search(unsubscribe_pattern, text)    if match:        return {            "type": "전화번호",            "value": match.group(1),            "action": "수신 거부"        }    return None# Define your schemaschema_ax = {    'properties': {    'message_info': {        'type': 'object',         'properties': {            'title': {'type': 'string', 'description': '광고 제목 - 원본 그대로 추출'},            'main_theme': {'type': 'string', 'description': '광고의 핵심 주제와 가치 제안을 명확하게 설명'},            'period': {'type': 'string', 'description': '이벤트/프로모션 기간 - 명시적으로 언급된 경우에만 추출, 없으면 "상시"로 설정'}        }    },    'purpose': {        'type': 'array',         'description': '광고의 주요 목적을 다음 중에서 선택(복수 가능): [상품 가입 유도, 대리점 방문 유도, 웹/앱 접속 유도, 이벤트 응모 유도, 혜택 안내, 쿠폰 제공 안내, 경품 제공 안내, 기타 정보 제공]'    },    'target': {        'type': 'array',         'items': {            'type': 'object',             'properties': {                'segment': {'type': 'string', 'description': '타겟 고객층 - 광고에서 명시적으로 언급하거나 암시한 대상 고객'},                'characteristics': {'type': 'string', 'description': '해당 타겟의 특성과 니즈'},                'priority': {'type': 'integer', 'description': '광고 내용에서의 중요도에 따른 타겟팅 우선순위 (1이 최우선)'}            }        }    },    'product': {        'type': 'array',         'items': {            'type': 'object',             'properties': {                'name': {'type': 'string', 'description': '광고하는 제품이나 서비스 이름 - "상품 후보 정보" 목록에서 일치하는 항목 모두 선택'},                'id': {'type': 'string', 'description': '제품/서비스 ID - "상품 후보 정보"에서 name에 일치하는 항목의 ID 선택'},                'category': {'type': 'string', 'description': '제품/서비스 카테고리 - "상품 후보 정보"에서 확인'},                'benefit': {                    'type': 'object',                     'properties': {                        'name': {'type': 'string', 'description': '제공되는 구체적인 혜택 내용 상세 설명'},                        'type': {'type': 'string', 'description': '혜택 유형: [할인, 쿠폰, 경품, 기타] 중에서 선택'}                    }                },                'conditions': {'type': 'string', 'description': '혜택/구매를 받기 위한 구체적인 조건'},                'action': {'type': 'string', 'description': '고객에게 기대하는 행동: [구매, 가입, 사용, 방문, 참여, 작성, 기타] 중에서 선택'}            }        }    },    'channel': {        'type': 'array',         'items': {            'type': 'object',             'properties': {                'type': {'type': 'string', 'description': '채널 종류: [URL, 전화번호, 앱, 대리점] 중에서 선택'},                'value': {'type': 'string', 'description': '실제 URL, 전화번호, 앱 이름, 대리점 이름 등 구체적 정보'},                'action': {'type': 'string', 'description': '채널 목적: [가입, 추가 정보, 문의, 수신, 수신 거부] 중에서 선택'},                'primary': {'type': 'boolean', 'description': '주요 채널 여부 - 광고의 핵심 연락 채널이면 true'},                'availability': {'type': 'string', 'description': '채널 이용 가능 시간/조건 - 언급이 없으면 "상시"로 설정'},                'benefit': {'type': 'string', 'description': '해당 채널 이용 시 특별 혜택'},                'store_code': {'type': 'string', 'description': "매장 코드 - tworldfriends.co.kr URL에서 D+숫자 9자리(D[0-9]{9}) 패턴의 코드 추출하여 대리점 채널에 설정"}            }        }    },    'metadata': {        'type': 'object',         'properties': {            'message_type': {'type': 'string', 'description': '메시지 유형 - 항상 "광고"로 설정'},            'target_response': {'type': 'string', 'description': '광고주가 고객으로부터 기대하는 반응 요약'},            'success_metrics': {'type': 'array', 'description': '이 캠페인의 성공을 측정하기 위한 지표들 추출'}        }    }}, 'required': ['purpose', 'target', 'product', 'channel', 'metadata'], 'objectType': 'object'}# Improved extraction guidance with clear focus on JSON outputextraction_guide = """### 분석 가이드 ###• 학습용 예시를 이용해서 few-shot 학습 기법을 사용합니다.• 광고 메시지에서 언급하는 모든 제품/서비스를 상품 정보 매핑에서 찾아 ID를 매칭하세요• 학습용 예시를 참고 정보로 사용하지 마세요. • 타겟 고객층과 혜택 내용을 명확히 식별하세요• 모든 채널 정보(URL, 전화번호 등)를 추출하세요• tworldfriends.co.kr URL에서 매장 코드(D+숫자 9자리)를 추출하세요• 명시되지 않은 정보는 적절히 추론하되, 추측은 최소화하세요### 응답 형식 ###순수 JSON만 반환하세요. 설명이나 추가 텍스트는 포함하지 마세요."""# Run the extraction with pre-processingmsg_text = """\t광고 제목:[SK텔레콤] 2월 0 day 혜택 안내\t광고 내용:(광고)[SKT] 2월 0 day 혜택 안내__[2월 10일(토) 혜택]_만 13~34세 고객이라면_베어유 모든 강의 14일 무료 수강 쿠폰 드립니다!_(선착순 3만 명 증정)_▶ 자세히 보기: http://t-mms.kr/t.do?m=#61&s=24589&a=&u=https://bit.ly/3SfBjjc__■ 에이닷 X T 멤버십 시크릿코드 이벤트_에이닷 T 멤버십 쿠폰함에 ‘에이닷이빵쏜닷’을 입력해보세요!_뚜레쥬르 데일리우유식빵 무료 쿠폰을 드립니다._▶ 시크릿코드 입력하러 가기: https://bit.ly/3HCUhLM__■ 문의: SKT 고객센터(1558, 무료)_무료 수신거부 1504"""# Pre-process to extract helpful informationstore_code = extract_store_code(msg_text)phone_numbers = extract_phone_numbers(msg_text)unsubscribe_info = extract_unsubscribe_info(msg_text)# Prepare relevant context - This assumes you have the tfidf_desc and few_shot_pdf variables definedsimilarities = cosine_similarity(            tfidf_desc.transform([msg_text]),            tfidf_matrix_desc        )[0]        # Assign similarities to dataframefew_shot_temp = few_shot_pdf.copy()  # Create a copy to avoid modifying originalfew_shot_temp['sim'] = similaritiesfew_shot_temp['rank'] = few_shot_temp["sim"].rank(method='min', ascending=False)# Get examples with clear JSON structurefew_shot_exm = []top_examples = few_shot_temp[    (few_shot_temp['rank'] >= 1) &     (few_shot_temp['rank'] <= 1)  # Increased to 3 examples].sort_values("rank")for _, r in top_examples.iterrows():    # Ensure examples show proper JSON format    example_json = r[f'res_{fs_model}']        # Clean up examples to show proper JSON format    if not (example_json.startswith('{') and example_json.endswith('}')):        json_match = re.search(r'({[\s\S]*})', example_json)        if json_match:            example_json = json_match.group(1)        example_json = extract_json_objects(example_json)[0]    # example_json.pop('product')    example_json = str(example_json)        few_shot_exm.append(f"""[학습용 광고 메세지_{_}]\t* 광고 제목:{r['msg_head']}\t* 광고 내용:{r['msg_body']}[학습용 정답 결과_{_}]\t{example_json}""")few_shot_exm_str = ("\n".join(few_shot_exm))# Enhanced entity matching - This assumes you have the find_entities_in_text and entity_list functions definedentity_list_trunc = [d for d in entity_list if d[1]['create_dt']>20230101]matches = find_entities_in_text(    msg_text,     entity_list_trunc,     min_similarity=65,  # Lower for more matches    ngram_size=3,          min_entity_length=2  # Shorter for Korean words)stop_item_names = ['SK텔레콤','혜택안내','수신거부','멤버십','150','고객센터']mdf = pd.DataFrame(matches).query("text.str.replace(' ', '') not in @stop_item_names")mdf['item_id'] = mdf['data'].apply(lambda x: x['item_id'])mdf['category'] = mdf['data'].apply(lambda x: x['category'])product_info = json.dumps(mdf.rename(columns={'text':'item_name_in_message','matched_entity':'item_name_in_voca'})[['item_name_in_message','item_name_in_voca','item_id','category']].drop_duplicates().to_dict(orient='records'), ensure_ascii=False)# product_info = mdf.rename(columns={'text':'item_name_in_message','matched_entity':'item_name_in_voca'})[['item_name_in_message','item_name_in_voca','item_id','category']].drop_duplicates().to_markdown() ## product_info = convert_to_custom_format(mdf.rename(columns={'text':'item_name_in_message','matched_entity':'item_name_in_voca'})[['item_name_in_message','item_id','category']].drop_duplicates().to_dict(orient='records'))# Improved context with structured auxiliary informationrag_context = "\n".join([f"\t* 상품 후보 정보: {product_info}",f"\t* 매장 코드 정보: '{store_code}'. 이 코드는 type='대리점'인 channel 객체의 store_code 필드에 설정해주세요.",f"\t* 수신 거부 정보: {unsubscribe_info}. 이 정보를 type='전화번호', action='수신 거부' channel 객체로 추가해주세요."])# Create the system message with clear JSON output requirementssystem_message = f"""당신은 SKT 캠페인 메시지에서 정확한 정보를 추출하는 전문가입니다. 아래 schema에 따라 광고 메시지를 분석하여 완전하고 정확한 JSON 객체를 생성해 주세요:{json.dumps(schema_ax, indent=2, ensure_ascii=False)}{extraction_guide}"""# Create message content with structured sectionsuser_message = f"""### 분석 대상 광고 메세지 ###{msg_text}### 관련 참고 정보 ###{rag_context}### 학습용 예시 ###{few_shot_exm_str}"""print('%'*15+" System Message "+'%'*15+"\n\n")print(system_message)print('%'*15+" User Message "+'%'*15+"\n\n")print(user_message)try:    # Use OpenAI's ChatCompletion with the current API format    response = client.chat.completions.create(        model="skt/a.x-3-lg",  # Or your preferred OpenAI model        messages=[            {"role": "system", "content": system_message},            {"role": "user", "content": user_message}        ],        temperature=0.0,        max_tokens=4000,        top_p=0.95,  # Reduces randomness        frequency_penalty=0.0,  # Avoid repetition in JSON        presence_penalty=0.0,        response_format={"type": "json_object"}  # Explicitly request JSON format    )        # Extract the JSON from the response    result_json_text = response.choices[0].message.content    json_objects = extract_json_objects(result_json_text)[0]    print("\n\n\n"+'%'*15+" LLM Result "+'%'*15+"\n\n")    print(json.dumps(json_objects, indent=4, ensure_ascii=False))            except Exception as e:    print(f"Error with API call: {e}")

In [ ]:
response = client.chat.completions.create(
        model="skt/a.x-3-lg",  # Or your preferred OpenAI model
        messages=[
            {"role": "user", "content": "how to make a good prompt to extract information from a text given examples for few-shot learning and context information for candidate entities to extract based on the following guides. \n\n1. Do not use the examples for few-shot learning as context information for RAG. "}
        ],
        temperature=0.0,
        max_tokens=4000,
        top_p=0.95,  # Reduces randomness
        frequency_penalty=0.0,  # Avoid repetition in JSON
        presence_penalty=0.0,
    )
response.choices[0].message.content

In [ ]:
import json
import re
import pandas as pd
from openai import OpenAI
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Initialize the OpenAI client
client = OpenAI(
    api_key = llm_api_key,
    base_url = llm_api_url
)
# Define your schema
schema_ax_prd = {
  "schema": {
    "type": "object",
    "properties": {
      "relationships": {
        "type": "array",
        "description": "메시지에서 추출한 상품 간의 관계 정보입니다.",
        "items": {
          "type": "object",
          "description": "하나의 상품과 다른 상품 간의 관계를 나타냅니다.",
          "properties": {
            "source_item": {
              "type": "object",
              "description": "관계의 주체가 되는 상품입니다. 보통 혜택이나 서비스를 제공하는 주체입니다.",
              "properties": {
                "id": {
                  "type": "string",
                  "description": "상품의 고유 식별자(ID)입니다."
                },
                "name": {
                  "type": "string",
                  "description": "상품의 이름입니다."
                },
                "description": {
                  "type": "string",
                  "description": "상품에 대한 간략한 설명입니다."
                }
              },
              "required": ["id", "name"]
            },
            "relation": {
              "type": "object",
              "description": "두 상품 간의 관계를 설명하는 객체입니다.",
              "properties": {
                "type": {
                  "type": "string",
                  "description": "관계의 유형입니다. '제공', '협업', '활용', '포함', '결합', '부가', '제휴' 등의 값을 가질 수 있습니다."
                },
                "description": {
                  "type": "string",
                  "description": "관계에 대한 상세 설명입니다."
                }
              },
              "required": ["type"]
            },
            "target_item": {
              "type": "object",
              "description": "관계의 대상이 되는 상품입니다. 보통 혜택이나 서비스를 받는 대상입니다.",
              "properties": {
                "id": {
                  "type": "string",
                  "description": "상품의 고유 식별자(ID)입니다."
                },
                "name": {
                  "type": "string",
                  "description": "상품의 이름입니다."
                },
                "description": {
                  "type": "string",
                  "description": "상품에 대한 간략한 설명입니다."
                }
              },
              "required": ["id", "name"]
            }
          },
          "required": ["source_item", "relation", "target_item"]
        }
      }
    },
    "required": ["relationships"]
  },
  "examples": {
    "relation_types": {
      "제공": "한 상품이 다른 상품에 혜택을 제공하는 관계입니다. 예: 0 day가 베어유 무료 수강 쿠폰을 제공",
      "협업": "두 상품이 공동으로 이벤트나 프로모션을 진행하는 관계입니다. 예: 에이닷과 T 멤버십이 시크릿코드 이벤트 진행",
      "활용": "한 상품이 다른 상품의 기능을 활용하는 관계입니다. 예: 에이닷이 T 멤버십 쿠폰함 기능을 활용",
      "포함": "한 상품이 다른 상품을 포함하는 관계입니다. 예: T 우주 패스가 YouTube Premium을 포함",
      "결합": "두 개 이상의 상품이 하나로 결합되어 새로운 가치를 창출하는 관계입니다. 예: 인터넷과 TV가 결합하여 할인 혜택 제공",
      "부가": "기본 상품에 추가적인 상품이나 서비스가 부가되는 관계입니다. 예: SK텔레콤에 T 올케어플러스5가 부가서비스로 추가",
      "제휴": "서로 다른 회사나 브랜드의 상품이 제휴 관계를 맺고 협력하는 관계입니다. 예: SK텔레콤과 Uber Taxi의 제휴"
    }
  }
}
# Improved extraction guidance with clear focus on JSON output
extraction_guide = """
### 분석 가이드 ###
순수 JSON만 반환하세요. 설명이나 추가 텍스트는 포함하지 마세요.
"""
pred_results = []
for m in mms_pdf.to_dict(orient='records'):
    # Run the extraction with pre-processing
  msg_text = f"""
\t광고 제목:{m['msg_nm']}
\t광고 내용:{m['mms_phrs']}
  """
  # Enhanced entity matching - This assumes you have the find_entities_in_text and entity_list functions defined
  entity_list_trunc = [d for d in entity_list if d[1]['create_dt']>20230101]
  matches = find_entities_in_text(
      msg_text,
      entity_list_trunc,
      min_similarity=65,  # Lower for more matches
      ngram_size=3,
      min_entity_length=2  # Shorter for Korean words
  )
  stop_item_names = ['SK텔레콤','혜택안내','수신거부','멤버십','150','고객센터']
  mdf = pd.DataFrame(matches).query("text.str.replace(' ', '') not in @stop_item_names")
  mdf['item_id'] = mdf['data'].apply(lambda x: x['item_id'])
  mdf['category'] = mdf['data'].apply(lambda x: x['category'])
  product_info = json.dumps(mdf.rename(columns={'text':'item_name_in_message','matched_entity':'item_name_in_voca'})[['item_name_in_message','item_name_in_voca','item_id','category']].drop_duplicates().to_dict(orient='records'), ensure_ascii=False)
  # product_info = mdf.rename(columns={'text':'item_name_in_message','matched_entity':'item_name_in_voca'})[['item_name_in_message','item_name_in_voca','item_id','category']].drop_duplicates().to_markdown() #
  # product_info = convert_to_custom_format(mdf.rename(columns={'text':'item_name_in_message','matched_entity':'item_name_in_voca'})[['item_name_in_message','item_id','category']].drop_duplicates().to_dict(orient='records'))
  # Improved context with structured auxiliary information
  rag_context = "\n".join([
  f"\t* 상품 후보 정보: {product_info}",
  ])
  # Create the system message with clear JSON output requirements
  system_message = f"""당신은 SKT 캠페인 메시지에서 정확한 정보를 추출하는 전문가입니다. 아래 schema에 따라 광고 메시지를 분석하여 아래 상품 후보 정보들의 관게를 추출해서 완전하고 정확한 JSON 객체를 생성해 주세요:
  {schema_ax_prd}
  """
  # Create message content with structured sections
  user_message = f"""### 분석 대상 광고 메세지 ###
  {msg_text}
  ### 관련 참고 정보 ###
  {rag_context}
  """
  # print('%'*15+" System Message "+'%'*15+"\n\n")
  # print(system_message)
  # print('%'*15+" User Message "+'%'*15+"\n\n")
  # print(user_message)
  try:
      # Use OpenAI's ChatCompletion with the current API format
      response = client.chat.completions.create(
          model="skt/a.x-3-lg",  # Or your preferred OpenAI model
          messages=[
              {"role": "system", "content": system_message},
              {"role": "user", "content": user_message}
          ],
          temperature=0.0,
          max_tokens=4000,
          top_p=0.95,  # Reduces randomness
          frequency_penalty=0.0,  # Avoid repetition in JSON
          presence_penalty=0.0,
          response_format={"type": "json_object"}  # Explicitly request JSON format
      )
      # Extract the JSON from the response
      result_json_text = response.choices[0].message.content
      json_objects = extract_json_objects(result_json_text)[0]
      pred_results.append(json_objects)
      # print("\n\n\n"+'%'*15+" LLM Result "+'%'*15+"\n\n")
      # print(json.dumps(json_objects, indent=4, ensure_ascii=False))
      # print(result_json_text)
  except Exception as e:
      print(f"Error with API call: {e}")
  break

In [ ]:
pred_results[0]

In [ ]:
def process_prompt(row):
    schema_ax = row['schema']
    msg_text = row['msg_text']
    system_message_ax = f"당신은 캠페인 메시지에서 정보를 추출하는 전문가입니다. 다음과 같은 schema에 부합하도록 JSON 개체를 만들어 주세요:\n {schema_ax}"
    # Create the chat completion
    response = client.chat.completions.create(
        model="skt/a.x-3-lg",  # or your specific model
        messages=[
            {"role": "system", "content": system_message_ax},
            {"role": "user", "content": msg_text}
        ],
        temperature=0.0
    )
    # print(response.choices[0].message.content)
    # Parse the response
    result_ax = extract_json_objects(response.choices[0].message.content)[0]
    return result_ax
results_ax_mul = []
with ThreadPoolExecutor(max_workers=len(prompt_list)) as executor:  # More threads for I/O-bound tasks
    results_ax_mul += list(executor.map(process_prompt, prompt_list))
result_ax = {}
for key in list(set(sum([list(d.keys()) for d in results_ax_mul], []))):
    lst = []
    for dic in results_ax_mul:
        if key in dic:
            lst.append(dic[key])
    dict_list = sum(lst,[])
    if is_list_of_dicts(dict_list):
        df = pd.DataFrame(dict_list)
        df = df.drop_duplicates()
        dict_list = df.to_dict('records')
    result_ax[key] = dict_list
print(json.dumps(result_ax, indent=4, ensure_ascii=False))


In [ ]:
for index in range(len(result_ax['product'])):
    item_name = result_ax['product'][index]['name'] #+ " " + result['광고 상품'][0]['상품 정보']
    # get_relevant_context(item_name, vectorstore, topk=20)['title']
    # item_pdf['sim'] = cosine_similarity(
    #     tfidf_name.transform([result_ax['광고 상품'][index]['상품명'] ]),
    #     tfidf_matrix_name
    # )[0]
    matches = find_entities_in_text(
        result_ax['product'][index]['name'],
        entity_list,
        min_similarity=70,  # Lower threshold for more matches
        ngram_size=3,       # Bigrams work well for Korean
        min_entity_length=3 # Only consider entities with 3+ characters
    )
    schema_res = {
    "ext_item_nm":{"type":"string","description":"기준 item의 name"},
    "item_id":{"type":"string","description":"가장 유사한 item의 id"},
    "item_nm":{"type":"string","description":"가장 유사한 item의 name"},
    "domain":{"type":"string","description":"해당 item의 도메인"},
    "reason":{"type":"string","description":"선택한 이유"}
    }
    text = f"""
    ---상품 정보---
    상품명: {result_ax['product'][index]['name']}
    상품 카테고리: {result_ax['product'][index]['category']}
    상품 정보: {result_ax['product'][index]['info']}
    ---원본 광고 메세지---
    {msg_text}
    ---후보 상품 리스트---
    {pd.DataFrame(matches)}
    """
    system_message_ax = f"""
    아래 광고 메세지에서 추출한 상품 정보와 후보 상품 리스트 중에서 광고 메세지의 맥락에 가장 부합하는 상품을 선택해라. 다음과 같은 schema에 참고하여 JSON 개체로 만들어 주세요:\n {schema_res}
    """
    # Create the chat completion
    response = client.chat.completions.create(
        model="skt/a.x-3-lg",  # or your specific model
        messages=[
            {"role": "system", "content": system_message_ax},
            {"role": "user", "content": text}
        ],
        temperature=0
    )
    res_ax = extract_json_objects((response.choices[0].message.content).replace("'",'"'))[0]
    print(json.dumps(res_ax, indent=4, ensure_ascii=False))

In [59]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
schema_cld = {'properties': {
    'message_info': {
        'type': 'object',
        'properties': {
            'title': {'type': 'string', 'description': '광고 제목 - 원본 그대로 추출'},
            'main_theme': {'type': 'string', 'description': '광고의 핵심 주제와 가치 제안을 명확하게 설명'},
            'period': {'type': 'string', 'description': '이벤트/프로모션 기간 - 명시적으로 언급된 경우에만 추출, 없으면 "상시"로 설정'}
        }
    },
    'purpose': {
        'type': 'array',
        'description': '광고의 주요 목적을 다음 중에서 선택(복수 가능): [상품 가입 유도, 대리점 방문 유도, 웹/앱 접속 유도, 이벤트 응모 유도, 할인 혜택 안내, 쿠폰 제공 안내, 경품 제공 안내, 기타 정보 제공]'
    },
    'target': {
        'type': 'array',
        'items': {
            'type': 'object',
            'properties': {
                'segment': {'type': 'string', 'description': '타겟 고객층 - 광고에서 명시적으로 언급하거나 암시한 대상 고객'},
                'characteristics': {'type': 'string', 'description': '해당 타겟의 특성과 니즈'},
                'priority': {'type': 'integer', 'description': '광고 내용에서의 중요도에 따른 타겟팅 우선순위 (1이 최우선)'}
            }
        }
    },
    'product': {
        'type': 'array',
        'items': {
            'type': 'object',
            'properties': {
                'name': {'type': 'string', 'description': '광고하는 제품이나 서비스 이름 - "상품 후보 정보" 목록에서 가장 일치하는 항목 선택'},
                'id': {'type': 'string', 'description': '제품/서비스 ID - "상품 후보 정보"에서 name에 일치하는 항목의 ID 선택'},
                'category': {'type': 'string', 'description': '제품/서비스 카테고리 - "상품 후보 정보"에서 확인'},
                'benefit': {
                    'type': 'object',
                    'properties': {
                        'name': {'type': 'string', 'description': '제공되는 구체적인 혜택 내용 상세 설명'},
                        'type': {'type': 'string', 'description': '혜택 유형: [할인, 쿠폰, 경품, 기타] 중에서 선택'}
                    }
                },
                'conditions': {'type': 'string', 'description': '혜택/구매를 받기 위한 구체적인 조건'},
                'action': {'type': 'string', 'description': '고객에게 기대하는 행동: [구매, 가입, 사용, 방문, 참여, 작성, 기타] 중에서 선택'}
            }
        }
    },
    'channel': {
        'type': 'array',
        'items': {
            'type': 'object',
            'properties': {
                'type': {'type': 'string', 'description': '채널 종류: [URL, 전화번호, 앱, 대리점] 중에서 선택'},
                'value': {'type': 'string', 'description': '실제 URL, 전화번호, 앱 이름, 대리점 이름 등 구체적 정보'},
                'action': {'type': 'string', 'description': '채널 목적: [가입, 추가 정보, 문의, 수신, 수신 거부] 중에서 선택'},
                'primary': {'type': 'boolean', 'description': '주요 채널 여부 - 광고의 핵심 연락 채널이면 true'},
                'availability': {'type': 'string', 'description': '채널 이용 가능 시간/조건 - 언급이 없으면 "상시"로 설정'},
                'benefit': {'type': 'string', 'description': '해당 채널 이용 시 특별 혜택'},
                'store_code': {'type': 'string', 'description': "매장 코드 - tworldfriends.co.kr URL에서 D+숫자 9자리(D[0-9]{9}) 패턴의 코드 추출하여 대리점 채널에 설정"}
            }
        }
    },
    'metadata': {
        'type': 'object',
        'properties': {
            'message_type': {'type': 'string', 'description': '메시지 유형 - 항상 "광고"로 설정'},
            'target_response': {'type': 'string', 'description': '광고주가 고객으로부터 기대하는 반응 요약'},
            'success_metrics': {'type': 'array', 'description': '이 캠페인의 성공을 측정하기 위한 지표들 추출'}
        }
    }
},
'required': ['purpose', 'target', 'product', 'channel', 'metadata'],
'objectType': 'object'}
# Improved extraction guidance
extraction_guide = """
### 분석 과정 ###
1. 먼저 광고 메시지를 전체적으로 읽고 주요 내용, 목적, 혜택, 타겟을 파악하세요.
2. "상품 후보 정보" 목록을 확인하여 광고에 언급된 제품/서비스와 가장 일치하는 항목을 선택하세요.
3. 각 제품별 혜택과 조건을 명확하게 구분하여 추출하세요.
4. 모든 채널 정보(URL, 전화번호, 대리점 등)를 빠짐없이 추출하고, tworldfriends.co.kr URL에서 매장 코드를 반드시 추출하세요.
5. 각 필드에 적합한 값을 선택할 때 광고 내용에 명시적으로 언급된 정보를 우선하고, 없는 경우에만 적절히 추론하세요.
6. 결과 JSON은 완전하고 정확해야 하며, schema에 정의된 모든 필수 필드를 포함해야 합니다.
### JSON 응답 형식 ###
응답은 설명 없이 순수한 JSON 형식으로만 제공하세요. 응답의 시작과 끝은 {}여야 합니다. 어떠한 추가 텍스트나 설명도 포함하지 마세요.
"""
# Custom parser to extract JSON from text response
class CustomJsonOutputParser(JsonOutputParser):
    def parse(self, text):
        # Try to find JSON in the response using regex
        json_match = re.search(r'({[\s\S]*})', text)
        if json_match:
            json_str = json_match.group(1)
            try:
                return json.loads(json_str)
            except json.JSONDecodeError:
                raise OutputParserException(f"Failed to parse JSON: {json_str}")
        else:
            raise OutputParserException(f"Could not find JSON in response: {text}")
# Create a prompt template with improved system message - emphasize JSON-only output
prompt_cld = ChatPromptTemplate.from_messages([
    ("system", """당신은 SKT 캠페인 메시지에서 정확한 정보를 추출하는 전문가입니다. 아래 schema에 따라 광고 메시지를 분석하여 완전하고 정확한 JSON 객체를 생성해 주세요:
{schema}
{extraction_guide}
=====참고 정보=====
{rag_context}
이제 다음 광고 메시지를 분석하여 schema에 맞는 완전한 JSON 객체를 생성해 주세요.
중요: 응답은 설명이나 추가 텍스트 없이 순수한 JSON 형식으로만 제공하세요. 응답은 '{{'로 시작하고 '}}'로 끝나야 합니다."""),
    ("human", "{text}")
])
# Create the custom parser
parser = CustomJsonOutputParser()
# Wrapper function to handle parsing failures
def extract_json_from_response(response_text):
    """Extract JSON from LLM response text even if it has explanatory text"""
    try:
        # First try to parse as pure JSON
        return json.loads(response_text)
    except json.JSONDecodeError:
        # If that fails, try to extract JSON using regex
        json_match = re.search(r'({[\s\S]*})', response_text)
        if json_match:
            json_str = json_match.group(1)
            try:
                return json.loads(json_str)
            except json.JSONDecodeError:
                print(f"Failed to parse extracted JSON pattern: {json_str}")
                return None
        else:
            print(f"Could not find JSON pattern in response: {response_text}")
            return None


In [65]:

schema_ax = {'properties': {
    'message_info': {
        'type': 'object',
        'properties': {
            'title': {'type': 'string', 'description': '광고 제목 - 원본 그대로 추출'},
            'main_theme': {'type': 'string', 'description': '광고의 핵심 주제와 가치 제안을 명확하게 설명'},
            'period': {'type': 'string', 'description': '이벤트/프로모션 기간 - 명시적으로 언급된 경우에만 추출, 없으면 "상시"로 설정'}
        }
    },
    'purpose': {
        'type': 'array',
        'description': '광고의 주요 목적을 다음 중에서 선택(복수 가능): [상품 가입 유도, 대리점 방문 유도, 웹/앱 접속 유도, 이벤트 응모 유도, 할인 혜택 안내, 쿠폰 제공 안내, 경품 제공 안내, 기타 정보 제공]'
    },
    'target': {
        'type': 'array',
        'items': {
            'type': 'object',
            'properties': {
                'segment': {'type': 'string', 'description': '타겟 고객층 - 광고에서 명시적으로 언급하거나 암시한 대상 고객'},
                'characteristics': {'type': 'string', 'description': '해당 타겟의 특성과 니즈'},
                'priority': {'type': 'integer', 'description': '광고 내용에서의 중요도에 따른 타겟팅 우선순위 (1이 최우선)'}
            }
        }
    },
    'product': {
        'type': 'array',
        'items': {
            'type': 'object',
            'properties': {
                'name': {'type': 'string', 'description': '광고하는 제품이나 서비스 이름 - "상품 후보 정보" 목록에서 가장 일치하는 항목 선택'},
                'id': {'type': 'string', 'description': '제품/서비스 ID - "상품 후보 정보"에서 name에 일치하는 항목의 ID 선택'},
                'category': {'type': 'string', 'description': '제품/서비스 카테고리 - "상품 후보 정보"에서 확인'},
                'benefit': {
                    'type': 'object',
                    'properties': {
                        'name': {'type': 'string', 'description': '제공되는 구체적인 혜택 내용 상세 설명'},
                        'type': {'type': 'string', 'description': '혜택 유형: [할인, 쿠폰, 경품, 기타] 중에서 선택'}
                    }
                },
                'conditions': {'type': 'string', 'description': '혜택/구매를 받기 위한 구체적인 조건'},
                'action': {'type': 'string', 'description': '고객에게 기대하는 행동: [구매, 가입, 사용, 방문, 참여, 작성, 기타] 중에서 선택'}
            }
        }
    },
    'channel': {
        'type': 'array',
        'items': {
            'type': 'object',
            'properties': {
                'type': {'type': 'string', 'description': '채널 종류: [URL, 전화번호, 앱, 대리점] 중에서 선택'},
                'value': {'type': 'string', 'description': '실제 URL, 전화번호, 앱 이름, 대리점 이름 등 구체적 정보'},
                'action': {'type': 'string', 'description': '채널 목적: [가입, 추가 정보, 문의, 수신, 수신 거부] 중에서 선택'},
                'primary': {'type': 'boolean', 'description': '주요 채널 여부 - 광고의 핵심 연락 채널이면 true'},
                'availability': {'type': 'string', 'description': '채널 이용 가능 시간/조건 - 언급이 없으면 "상시"로 설정'},
                'benefit': {'type': 'string', 'description': '해당 채널 이용 시 특별 혜택'},
                'store_code': {'type': 'string', 'description': "매장 코드 - tworldfriends.co.kr URL에서 D+숫자 9자리(D[0-9]{9}) 패턴의 코드 추출하여 대리점 채널에 설정"}
            }
        }
    },
    'metadata': {
        'type': 'object',
        'properties': {
            'message_type': {'type': 'string', 'description': '메시지 유형 - 항상 "광고"로 설정'},
            'target_response': {'type': 'string', 'description': '광고주가 고객으로부터 기대하는 반응 요약'},
            'success_metrics': {'type': 'array', 'description': '이 캠페인의 성공을 측정하기 위한 지표들 추출'}
        }
    }
},
'required': ['purpose', 'target', 'product', 'channel', 'metadata'],
'objectType': 'object'}
# Improved extraction guidance with clear focus on JSON output
extraction_guide = """
### 분석 과정 ###
1. 먼저 광고 메시지를 전체적으로 읽고 주요 내용, 목적, 혜택, 타겟을 파악하세요.
2. "상품 후보 정보" 목록을 확인하여 광고에 언급된 제품/서비스와 가장 일치하는 항목을 선택하세요.
3. 각 제품별 혜택과 조건을 명확하게 구분하여 추출하세요.
4. 모든 채널 정보(URL, 전화번호, 대리점 등)를 빠짐없이 추출하고, tworldfriends.co.kr URL에서 매장 코드를 반드시 추출하세요.
5. 각 필드에 적합한 값을 선택할 때 광고 내용에 명시적으로 언급된 정보를 우선하고, 없는 경우에만 적절히 추론하세요.
6. 결과 JSON은 완전하고 정확해야 하며, schema에 정의된 모든 필수 필드를 포함해야 합니다.
### JSON 응답 형식 ###
응답은 설명 없이 순수한 JSON 형식으로만 제공하세요. 설명이나 추가 텍스트를 포함하지 마세요.
"""


In [66]:
def get_llm_result_by_fsl(msg_text, schema_ax, extraction_guide, rag_context, few_shot_exm_str, res_dic_empt, row, fs_model, num_fs=1):
    try:
                # Create the system message with clear JSON output requirements
        system_message = f"""당신은 SKT 캠페인 메시지에서 정확한 정보를 추출하는 전문가입니다. 아래 schema에 따라 광고 메시지를 분석하여 완전하고 정확한 JSON 객체를 생성해 주세요:
        {json.dumps(schema_ax, indent=2, ensure_ascii=False)}
        {extraction_guide}
        중요: 응답은 설명이나 추가 텍스트 없이 순수한 JSON 형식으로만 제공하세요. 응답은 '{{'로 시작하고 '}}'로 끝나야 합니다.
        """
        # Create message content with structured sections
        user_message = f"""=====분석 대상 광고 메세지=====
        {msg_text}
        =====관련 참고 정보=====
        {rag_context}
        =====학습용 예시=====
        {few_shot_exm_str}
        """
        response = client.chat.completions.create(
            model="skt/a.x-3-lg",  # Or your preferred OpenAI model
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ],
            temperature=0.0,
            response_format={"type": "json_object"}  # Explicitly request JSON format
        )
        # Extract the JSON from the response
        result_json_text = response.choices[0].message.content
        result = extract_json_objects(result_json_text)[0]
        row[f'res_ax_f{num_fs}'] = result
    except Exception as e:
        print(f"Error message when calling ax_f{num_fs}: \n{result_json_text}")
        row[f'res_ax_f{num_fs}'] = res_dic_empt#.replace('\n','')
    return row
                        

In [ ]:
import pandas as pdimport jsonbuild_few_shot = Falsejob_resume = Falseref_model = "cld37"n_sample_fsl = 500ref_model_fs = "gpt"max_workers = 1if ref_model == "cld35":    llm_modl = llm_cld35elif ref_model == "cld37":    llm_modl = llm_cld37else:    llm_modl = llm_chatn_fsl_sta = 2n_fsl_end = 2# Create the parserparser = JsonOutputParser()extraction_chain = prompt_cld | llm_modl | parserextraction_chain_cld35 = prompt_cld | llm_cld35 | parserfew_shot_pdf = pd.read_csv(f"./data/few_shot_data_gpt_250415_rd_500.csv")few_shot_pdf['index'] = few_shot_pdf['index'].astype('str')few_shot_pdf['offer_dt'] = few_shot_pdf['offer_dt'].astype('str')tfidf_desc = TfidfVectorizer(            analyzer='char',            ngram_range=(3,5),            min_df=1,            # stop_words=None,  # Don't use stop words initially            # token_pattern=r'(?u)\b\w+\b',  # Match any word character (more permissive)            # strip_accents='unicode',  # Handle accented characters        )tfidf_matrix_desc = tfidf_desc.fit_transform(few_shot_pdf["msg_body"])rep_dict = {'[SK텔레콤]':'','[SKT]':''}special_symbols_to_remove = {'‘':'','’':'',':':'',"'":'','"':'','_':'\n'}def process_prompt(row, few_shot_pdf, ref_model):    res_dic_empt = {        "purpose": "",        "target": "",        "product": "",        "channel": ""    }        head = row['head']    body = row['body']        head = replace_strings(head, special_symbols_to_remove)    body = replace_strings(body, special_symbols_to_remove)            #     for k,v in special_symbols_to_remove.items():#         head = head.replace(k,v)#         body = body.replace(k,v)        msg_text = f"""        광고 제목: {head}        광고 내용: {body}    """        #     rag_context = get_relevant_context(text, vectorstore, topk=20)['title']    #     row['res_rag'] = rag_context    # Enhanced entity matching    matches = find_entities_in_text(        msg_text,         entity_list,         min_similarity=65,        ngram_size=4,        min_entity_length=2    )    product_info = ", ".join(set([match['text'] for match in matches]))    mdf = pd.DataFrame(matches)    mdf['item_id'] = mdf['data'].apply(lambda x: x['item_id'])    mdf['category'] = mdf['data'].apply(lambda x: x['category'])    product_info = json.dumps(mdf.rename(columns={'text':'item_name_in_message','matched_entity':'item_name_in_voca'})[['item_name_in_message','item_name_in_voca','item_id','category']].drop_duplicates().to_dict(orient='records'), ensure_ascii=False)    if not build_few_shot:        store_code = extract_store_code(msg_text)        phone_numbers = extract_phone_numbers(msg_text)        unsubscribe_info = extract_unsubscribe_info(msg_text)        # Improved context with structured auxiliary information        rag_context = "\n".join([            f"\t상품 후보 정보: {product_info}",            f"\t매장 코드 정보: '{store_code}'. 이 코드는 type='대리점'인 channel 객체의 store_code 필드에 설정해주세요.",            f"\t수신 거부 정보: {unsubscribe_info}. 이 정보를 type='전화번호', action='수신 거부' channel 객체로 추가해주세요."        ])                # Calculate similarities        similarities = cosine_similarity(            tfidf_desc.transform([msg_text]),            tfidf_matrix_desc        )[0]                # Assign similarities to dataframe        few_shot_temp = few_shot_pdf.copy()  # Create a copy to avoid modifying original        few_shot_temp['sim'] = similarities        few_shot_temp['rank'] = few_shot_temp["sim"].rank(method='min', ascending=False)        # Get examples with better structure - make sure each example shows JSON only in the result        few_shot_exm = []        top_examples = few_shot_temp[            (few_shot_temp['rank'] >= 1) &             (few_shot_temp['rank'] <= 2)        ].sort_values("rank")        for _, r in top_examples.iterrows():            # Ensure the examples show proper JSON-only outputs            example_json = r[f'res_{ref_model_fs}']                        # If the example contains explanatory text, try to extract just the JSON            if not (example_json.startswith('{') and example_json.endswith('}')):                json_match = re.search(r'({[\s\S]*})', example_json)                if json_match:                    example_json = json_match.group(1)                        few_shot_exm.append(f"""        \t[학습용 광고 메세지_{_}]        \t\t광고 제목:{r['msg_head']}        \t\t광고 내용:{r['msg_body']}        \t[학습용 정답 결과_{_}]        \t\t{example_json}""")        few_shot_exm_str = "\n".join(few_shot_exm)                for nf in range(n_fsl_sta, n_fsl_end+1):            row = get_llm_result_by_fsl(msg_text, schema_ax, extraction_guide, rag_context, few_shot_exm_str, res_dic_empt, row, ref_model, num_fs=nf)        try:        rag_context = f"\t상품 후보 정보: {product_info}"        result = extraction_chain.invoke({            "text": msg_text,             "schema": str(schema_cld),             "rag_context": rag_context,             # "few_shot_exm_str": few_shot_exm_str,            "extraction_guide": extraction_guide        })        # result = json.dumps(result, indent=4, ensure_ascii=False)        row[f'res_{ref_model}'] = result            except Exception as e:        try:            result = extract_json_objects(e)[0]            row[f'res_{ref_model}'] = result        except Exception as e2:            print(f"Error message when calling {ref_model}: \n{e2}")            row[f'res_{ref_model}'] = res_dic_empt    # try:    #     result = extraction_chain_cld35.invoke({"text": msg_text, "schema": str(schema_cld)    #                                       # , "rag_context":rag_context    #                                      })    #     # result = json.dumps(result, indent=4, ensure_ascii=False)    #     row[f'res_cld35'] = result            # except Exception as e:    #     try:    #         result = extract_json_objects(e)[0]    #         row[f'res_cld35'] = result    #     except Exception as e2:    #         print(f"Error message when calling cld35: \n{e2}")    #         row[f'res_cld35'] = res_dic_empt    return rowstart_time = time.time()import randomfrom functools import partial# mdf = mms_pdf.rename(columns={'msg_nm':'head','mms_phrs':'body'}).to_dict("records")if not job_resume:    test_pdf_list = []if build_few_shot:    mdf = (mms_pdf    # .merge(pd.DataFrame(test_pdf_list)[['index',f'res_{ref_model}']],on='index',how='left').query(f"res_{ref_model}.isna()").drop(f"res_{ref_model}",axis=1)    .rename(columns={'msg_nm':'head','mms_phrs':'body'}).sample(n=(n_sample_fsl-len(test_pdf_list))).to_dict("records"))else:    mdf = mms_pdf.query("offer_dt>='20240101'").merge(few_shot_pdf[['index',f'res_{ref_model_fs}']],on='index',how='left').query(f"res_{ref_model_fs}.isna()").drop(f"res_{ref_model_fs}",axis=1).rename(columns={'msg_nm':'head','mms_phrs':'body'})    mdf = mdf.sample(n=5).to_dict("records")    # mdf = pd.read_excel("./data/mms_extraction_data_250415.xlsx").rename(columns={'msg_head':'head','msg_body':'body'}).to_dict("records")# Create a partial function with few_shot_pdfprocess_prompt_with_pdf = partial(process_prompt, few_shot_pdf=few_shot_pdf, ref_model=ref_model)if max_workers>1:    print("Muli-processing starts")    with ThreadPoolExecutor(max_workers=max_workers) as executor:        test_pdf_list += list(executor.map(process_prompt_with_pdf, mdf))else:    print("Single-processing starts")    for row in mdf:        processed_row = process_prompt_with_pdf(row)        test_pdf_list.append(processed_row)        if len(test_pdf_list)%10==0:            print(len(test_pdf_list))end_time = time.time()elapsed_time = end_time - start_timeprint(f"Execution took {elapsed_time:.4f} seconds")    # test_pdf = pd.DataFrame(test_pdf_list)

In [ ]:
rdf = (pd.DataFrame(test_pdf_list).rename(columns={'head':'msg_head','body':'msg_body'}).drop(['msg'], axis=1))
rdf['res_cld37'] = rdf['res_cld37'].apply(lambda x:json.dumps(x, indent=4, ensure_ascii=False))
# print(rdf[''][0])
print(rdf['msg_head'][0], "\n",
rdf['msg_body'][0])
# msg_text

In [100]:
pd.set_option('display.max_colwidth', 1000)
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d%H")
if build_few_shot:
    rdf = (pd.DataFrame(test_pdf_list).rename(columns={'head':'msg_head','body':'msg_body'}).drop(['msg'], axis=1))
    rdf = rdf[(rdf[f"res_{ref_model}"].apply(lambda x: x.get('purpose') != ''))]
    rdf.to_csv(f"./data/few_shot_data_{ref_model}_{timestamp}_rd_{n_sample_fsl}.csv", index=False)
else:
    rdf = (pd.DataFrame(test_pdf_list).rename(columns={'head':'msg_head','body':'msg_body',f'res_ax_f{n_fsl_end}':f'res_ax_{ref_model}'}))
    rdf = rdf[[c for c in rdf.columns if not (c.startswith('fsl_') or c.startswith('cot_'))]]
    rdf['msg_body'] = rdf['msg_body'].apply(remove_control_characters)
    rdf = rdf[(rdf[f'res_ax_{ref_model}'].apply(lambda x: x.get('purpose') != '')) & (rdf[f"res_{ref_model}"].apply(lambda x: x.get('purpose') != ''))]
    rdf[f'res_ax_{ref_model}'] = rdf[f'res_ax_{ref_model}'].apply(lambda x:json.dumps(x, indent=4, ensure_ascii=False))
    rdf[f"res_{ref_model}"] = rdf[f"res_{ref_model}"].apply(lambda x:json.dumps(x, indent=4, ensure_ascii=False))
    rdf.sort_values('offer_dt').to_excel(f"./data/mms_extraction_data_{timestamp}.xlsx", index=False, engine='openpyxl')


In [ ]:
rdf = (pd.DataFrame(test_pdf_list).rename(columns={'head':'msg_head','body':'msg_body'}).drop(['msg'], axis=1))
for nf in (list(range(n_fsl_sta, n_fsl_end+1))):
    rdf = rdf[(rdf[f'res_ax_f{nf}'].apply(lambda x: x.get('purpose') != ''))]
    # rdf = rdf[(rdf['res_ax_f'].apply(lambda x: x.get('purpose') != '')) & (rdf['res_ax_n'].apply(lambda x: x.get('purpose') != ''))]
sdf = rdf.copy()
# # sdf = pd.read_pickle("/home/skinet/myfiles/tos_ace/data/mms_extraction_data_vl_250403_01.pkl")
for nf in (list(range(n_fsl_sta, n_fsl_end+1))):
    sdf[f'sim_f{nf}'] = sdf.apply(lambda x : calculate_dictionary_similarity(x[f'res_ax_f{nf}'], x[f'res_{ref_model}']), axis=1)
    # sdf[f'sim_f{nf}'] = sdf.apply(lambda x : calculate_dictionary_similarity(x[f'res_ax_f{nf}'], x[f'res_cld35']), axis=1)
    print(nf, "\n", pd.json_normalize(sdf[f'sim_f{nf}']).mean())


In [54]:
def json_to_custom_string(json_obj, indent=0):    result = []    indent_str = "  " * indent        if isinstance(json_obj, dict):        for key, value in json_obj.items():            if isinstance(value, (dict, list)):                result.append(f"{indent_str}{key}:")                result.append(json_to_custom_string(value, indent + 1))            else:                result.append(f"{indent_str}{key}: {value}")    elif isinstance(json_obj, list):        for item in json_obj:            if isinstance(item, (dict, list)):                result.append(json_to_custom_string(item, indent))            else:                result.append(f"{indent_str}- {item}")        return "\n".join(result)# json_to_custom_string(parsed_json)

In [ ]:
!pip3.12 install flask